This notebook shows the effects of using the long-form text consolidation and summarization functions I made in the consolidation-comparison.ipynb notebook on the Bible. I ran them on the whole Bible, the New Testament, the Old Testament, and each book. I've included the description of those techniques below for the sake of completeness. I did this purely out of curiosity; this is not meant to serve as any form of conclusive analysis.

# Methods

## TF-IDF

Term frequency-inverse document frequency gives high weights to words that are present in a given document (term frequency), but rare across the entire corpus (inverse document frequency). I wanted to explore the utility of using this techinque for consolidating long texts. I used the follow process:

1. Turn text into a list of sentences.
2. Create the sparse TF-IDF matrix from these sentences. Each sentence is a document, and the full collection of sentences is the corpus.
3. Pool the vectors in this matrix using either max or sum pooling.
    - Max pooling captures the maximum TF-IDF score for each word. Broadly speaking, this vector represents the text's most important topics.
    - Sum pooling sums all TF-IDF scores for each word. Broadly speaking, this vector represents the text's recurring themes.
5. Use MMR to select sentences based on their TF-IDF vectors.

## Transformers

This approach uses sentence transformers in fundamentally the same process as I used for TF-IDF consolidation.

1. Chunk text into either sentences or fixed-size chunks.
2. Embed text chunks.
3. Mean pool embeddings.
4. Use MMR to select sentences based on their embeddings.

NB: `transformers_consolidation()` contains options for length weighted mean pooling and semantic centrality pooling, which aren't implemented in this notebook, but were implemented in another notebook not in this repository. Length weighted mean pooling finds the average of the vectors weighted by the length of the sentence each vector represents. Semantic centrality pooling weights each embedding by its average cosine similarity to all other embeddings. 

## Hugging Face Summarization Pipeline

The approach iteratively summarizes long-form text using Hugging Face's summarization pipeline. 

1. Determine token limit of summarization model.
2. Chunk text into pieces ~50% of model's token limit.
3. Summarize each chunk to 30-90% of its original length.
4. Concatentate chunks.
5. Repeat chunking and summarization process of concatenated text until concatenated text is within defined limits.

NB: `summarization_pipeline_consolidation()` contains options for paragraph chunking and semantic chunking, which aren't implemented in this notebook. Where `semantic_chunking()` is implemnented, it uses LangChain's SemanticChunker (`from langchain_experimental.text_splitter import SemanticChunker`). 

# Observations

There's a lot of text in this notebook, so I haven't read through all of it yet, but here are some of my more immediate observations.

- TF-IDF consolidation with max pooling produced identical results for the whole Bible and the Old Testament.
- Transformers consolidation with chunk pooling produced very similar results for the whole Bible and the Old Testament. The only difference was a single sentence, "And Araunah said unto David, Let my lord the king take and offer up what seemeth good unto him: behold, here be oxen for burnt sacrifice, and threshing instruments and other instruments of the oxen for wood," which is present in the consolidation for the whole Bible but not the Old Testament.
- Much of the text returned by `summarization_pipeline_consolidation()` is similar between the whole Bible and the Old Testament. The whole Bible summarization has several more sentences at the end, however.

I expect that the previous three observations simply result from the fact that the Old Testament is a much larger portion of the Bible than the New Testament. Changing the MMR lambda value may produce different results.

Part of the inspiration for this project was curiosity about what would happen if I tried to condense the Bible into a single, discourse coherent paragraph. However, the summarization pipeline results overall seem too noisy to be usable in any meaningful way. They're better for the individual books, espeically the shorter ones, but not for the full testaments and the whole Bible. Iteratively summarizing text like I did opens up too many opportunities for artifacts to appear, which they certainly did. I'm sure I'd get usable results if I used one of the big LLMs (ChatGPT, Claude, etc.) or did some careful model selection and parameter tuning. 

I first tried to use the pegasus-x-large model because it produced such good results in the consolidation_comparison.ipynb notebook before it devolved into repetition. I thought that, with some parameter tuning, it could give me what I was hoping for. However, running it locally on the entire Bible proved too resource intensive (i.e. it was taking too long and I got impatient), so I opted for sshleifer/distilbart-cnn-12-6, which produced sufficiently passable results in the other notebook.

One thing to keep in mind with these results is that, due to the nature of NLP, these results are semantically important, but not necessarily theologically important. Studying the overlap between semantic and theological importance could be an an interesting project for later.

# Imports and Functions

In [1]:
import numpy as np
import ebooklib
from bs4 import BeautifulSoup
from ebooklib import epub
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoConfig, AutoTokenizer, pipeline
from transformers import logging

import math
import re
import warnings
import requests

logging.set_verbosity_error()

C:\MyStuff\Applications\Productivity\Programming\Python\jupyter-env-313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Bible Parsing Functions

In [2]:
def get_book_text(book: str, bible: dict) -> str:
    """
    Extract and concatenate all chapter text from a specific book of the Bible.
    
    Args:
        book (str): Name of the biblical book (e.g., 'Genesis', 'Luke')
        bible (dict): Dictionary with book names as keys and nested dictionaries as values.
            Each nested dictionary has chapter numbers (int) as keys and chapter text (str) as values.
            Structure: {book_name: {chapter_number: chapter_text}}
    
    Returns:
        str: Complete text of the specified book with chapters concatenated and 
            separated by newlines
    
    Raises:
        ValueError: If the specified book is not found in the Bible dictionary
    """
    if book not in bible:
        raise ValueError(f"Book '{book}' not found in Bible")
    
    return '\n'.join(text for _, text in bible[book].items())


def get_chapter_text(book: str, chapter: int, bible: dict) -> str:
    """
    Extract the text from a specific chapter of a biblical book.
    
    Args:
        book (str): Name of the biblical book (e.g., 'Genesis', 'Luke')
        chapter (int): Chapter number (e.g., 1, 2, 3)
        bible (dict): Dictionary with book names as keys and nested dictionaries as values.
            Each nested dictionary has chapter numbers (int) as keys and chapter text (str) as values.
            Structure: {book_name: {chapter_number: chapter_text}}
    
    Returns:
        str: Text content of the specified chapter
    
    Raises:
        ValueError: If the specified book is not found in the Bible dictionary
        ValueError: If the specified chapter is not found in the book
    """
    if book not in bible:
        raise ValueError(f"Book '{book}' not found in Bible")

    for chapter_title, chapter_text in bible[book]:
        if chapter_title == chapter:
            return chapter_text
    
    raise ValueError(f"Chapter '{chapter}' not found in book '{book}'. "
                     f"'{book}' has {len(bible[book])} chapters.")

## NLP Functions

In [3]:
def mean_pooling(embeddings: np.ndarray) -> np.ndarray:
    """
    Compute the mean pooling of embeddings.
    
    Args:
        embeddings (np.ndarray): Input embeddings array
       
    Returns:
        np.ndarray: Mean-pooled vector
    """
    text_vector = np.mean(embeddings, axis=0)

    return text_vector
    

def fixed_size_chunking(text: str, model_name: str = 'sentence-transformers/all-MiniLM-L6-v2', doc_size: int = None, full_sentences: bool = True) -> list[str]:
    """
    Split text into fixed-size chunks based on token count.

    When full_sentences=True, tries to keep sentences intact but will split oversized
    sentences. When False, splits text purely by token count regardless of boundaries.
    
    Args:
        text (str): Input text to chunk
        model_name (str): HuggingFace model name for tokenizer. 
            Defaults to 'sentence-transformers/all-MiniLM-L6-v2'.
        doc_size (int | None): Maximum tokens per chunk. If None, uses model's max 
            length with a buffer for safety. Defaults to None.
        full_sentences (bool): If True, preserves sentence boundaries when possible.
            Defaults to True.
       
    Returns:
        list[str]: List of text chunks, each within the token limit
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if doc_size is None:
        doc_size = get_model_max_length(model_name)[0] - 50  # -50 b/c sometimes I reach token limits without it

    docs = []
    
    if full_sentences:
        sentences = sent_tokenize(text)

        encoded_sentences = [tokenizer.encode(sent) for sent in sentences]
        current_tokens = 0
        current_chunk = []
        for sent, encoded_sent in zip(sentences, encoded_sentences):
            # If adding this sentence would exceed the limit, finalize current chunk and reset everything
            if len(encoded_sent) + current_tokens > doc_size and current_chunk:
                chunk_text = ' '.join(current_chunk)
                docs.append(chunk_text)
                current_chunk = []
                current_tokens = 0

            # If a single sentence exceeds doc_size, split it up
            if len(encoded_sent) > doc_size:
                if current_chunk:
                    # First, add any accumulated sentences
                    chunk_text = ' '.join(current_chunk)
                    docs.append(chunk_text)
                    current_chunk = []
                    current_tokens = 0

                for i in range(0, len(encoded_sent), doc_size):
                    chunk_tokens = encoded_sent[i:i + doc_size]
                    doc_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                    docs.append(doc_text)
            else:
                # Add sentence to current chunk
                current_chunk.append(sent)
                current_tokens += len(encoded_sent)

        if current_chunk:
            chunk_text = ' '.join(current_chunk)
            docs.append(chunk_text)
            
    else:
        tokens = tokenizer.encode(text)        
        
        for i in range(0, len(tokens), doc_size):
            chunk_tokens = tokens[i:i + doc_size]
            doc_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            docs.append(doc_text)
    
    return docs

In [4]:
def get_model_max_length(model_name: str) -> tuple[int, str]:
    """
    Get the maximum sequence length for a given model.

    Tries multiple common config attributes in order of preference, falls back to 
    tokenizer.model_max_length, then defaults to 512 if nothing reasonable is found.
    Filters out unreasonably large values (>1M) that are likely placeholder defaults.
    
    Args:
        model_name (str): HuggingFace model identifier
       
    Returns:
        tuple[int, str]: A tuple containing:
            - max_length (int): Maximum sequence length
            - source_attribute (str): Which attribute was used to determine the length
            
    Raises:
        Warning: If model cannot be loaded or no reliable max length found
    """
    try:
        max_reasonable_length = 1_000_000
        
        config = AutoConfig.from_pretrained(model_name)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        # List of possible attributes that indicate max length
        # Order matters - more specific attributes first
        length_attributes = [
            'max_source_positions',     # BART, Pegasus
            'max_position_embeddings',  # BERT, GPT, many others
            'n_positions',              # GPT-2, GPT-J
            'max_sequence_length',      # Some custom models
            'model_max_length',         # Tokenizer (fallback)
            'max_len',                  # Some older models
        ]
        
        found_length = None
        used_attribute = None
        
        # Check config first
        for attr in length_attributes[:-1]:  # Exclude model_max_length for now
            if hasattr(config, attr):
                value = getattr(config, attr)
                # Filter out unreasonably large values (likely defaults)
                if value and value < max_reasonable_length:
                    found_length = value
                    used_attribute = f"config.{attr}"
                    break
        
        # If no reasonable value found in config, check tokenizer as fallback
        if found_length is None:
            tokenizer_max = tokenizer.model_max_length
            if tokenizer_max < max_reasonable_length:
                found_length = tokenizer_max
                used_attribute = "tokenizer.model_max_length"
            else:
                # Last resort - use a conservative default
                found_length = 512
                used_attribute = "default (no reliable max found)"
                warnings.warn(f"Could not determine max length for {model_name}, using conservative default of 512")
        
        return found_length, used_attribute
        
    except Exception as e:
        warnings.warn(f"Error getting max length for {model_name}: {e}. Using default of 512.")
        return 512, "default (error occurred)"

In [5]:
def mmr(doc_embeddings: np.ndarray, query_embedding: np.ndarray, max_tokens: int, chunked_sentences: list[tuple], lambda_param: float = 0.5) -> list:
    """
    Maximal Marginal Relevance with token budget constraint.
    
    Args:
        doc_embeddings (np.ndarray): Document embeddings matrix of shape (n_docs, embedding_dim)
        query_embedding (np.ndarray): Query embedding vector of shape (embedding_dim,)
        max_tokens (int): Maximum token budget for selected sentences
        chunked_sentences (list[tuple]): List of (text, token_list) tuples where 
            text is the sentence string and token_list is the tokenized representation
        lambda_param (float): Relevance vs diversity trade-off parameter. 
            1.0 = pure relevance, 0.0 = pure diversity. Defaults to 0.5.
    
    Returns:
        list[tuple]: Selected sentences as (text, token_list) tuples within token budget
    """
    query_similarities = cosine_similarity(doc_embeddings, [query_embedding]).flatten()

    selected = []
    remaining = list(range(len(doc_embeddings)))
    token_count = 0
    
    while remaining and token_count < max_tokens:
        if not selected:
            idx = np.argmax(query_similarities)
        else:
            mmr_scores = []
            for i in remaining:
                max_selected_similarity = np.max(cosine_similarity([doc_embeddings[i]], doc_embeddings[selected]))
                mmr = lambda_param * query_similarities[i] - (1 - lambda_param) * max_selected_similarity
                mmr_scores.append(mmr)

            best_idx = np.argmax(mmr_scores)            
            idx = remaining[best_idx]  

        # Check if adding this sentence would exceed token limit
        sentence_tokens = len(chunked_sentences[idx][1])
        if token_count + sentence_tokens > max_tokens:
            break
            
        selected.append(idx)
        remaining.remove(idx)
        token_count += sentence_tokens

    return [chunked_sentences[i] for i in selected]

In [6]:
def tfidf_consolidation(text: str, model_name: str = 'sentence-transformers/all-MiniLM-L6-v2', output: str = 'text', pooling: str = 'max', max_tokens: int = None, mmr_lambda: float = 0.5) -> list:
    """
    Consolidate text by selecting most important sentences using TF-IDF and MMR.

    Splits text into sentences, calculates TF-IDF vectors for each sentence, creates
    a document centroid using pooling, then applies Maximal Marginal Relevance (MMR)
    to select sentences that balance relevance to the document centroid with diversity.
    
    Args:
        text (str): Input text to consolidate
        model_name (str): HuggingFace model name used to determine token limits and 
            tokenization. Defaults to 'sentence-transformers/all-MiniLM-L6-v2'.
        output (str): Output format - 'tokens' returns token lists, 'text' returns 
            strings. Defaults to 'text'.
        pooling (str): Pooling method for document centroid - 'max' (element-wise 
            maximum) or 'sum' (element-wise sum). Defaults to 'max'.
        max_tokens (int | None): Maximum tokens in output. If None, uses model's 
            maximum length. Defaults to None.
        mmr_lambda (float): MMR relevance vs diversity trade-off parameter 
            (1.0=pure relevance, 0.0=pure diversity). Defaults to 0.5.
       
    Returns:
        list[list[int]] | list[str]: Selected sentences as token lists or text strings 
            based on output parameter
            
    Raises:
        ValueError: If no sentences found in input text
    """
    # Initialize tokenizer from model name
    tokenizer = AutoTokenizer.from_pretrained(model_name)
        
    # Auto-detect model's token limit
    if max_tokens is None:
        max_tokens = get_model_max_length(model_name)[0]

    # Split text into sentences (NB: does not split along newlines)
    sentences = sent_tokenize(text)
    if not sentences:
        raise ValueError("No sentences found in text.")
    encoded_sentences = [(sentence, tokenizer.encode(sentence)) for sentence in sentences]

    # Check if consolidation is needed
    text_tokens = sum(len(item[1]) for item in encoded_sentences)
    if text_tokens <= max_tokens: # No consolidation needed
        if output == 'tokens':
            return [item[1] for item in encoded_sentences]
        elif output == 'text':
            return [item[0] for item in encoded_sentences]
        else:
            warnings.warn('Invalid output. Defaulting to tokens.')
            return [item[1] for item in encoded_sentences]
    
    chunked_sentences = []
    for sentence, encoded in encoded_sentences:
        if len(encoded) <= max_tokens:            
            chunked_sentences.append((sentence, encoded))
        else:
            # Break apart sentences that are longer than max_tokens
            # TODO this approach is inefficient, but works for now
            for i in range(0, len(encoded), max_tokens):
                segment_text = tokenizer.decode(encoded[i:i + max_tokens], skip_special_tokens=True, clean_up_tokenization_spaces=True)
                segment_tokens = encoded[i:i + max_tokens]
                chunked_sentences.append((segment_text, segment_tokens))
    
    # Calculate sentence importance using TF-IDF
    vectorizer = TfidfVectorizer(stop_words='english')
    
    # Create document-term matrix (each sentence is a document)
    try:
        sentence_vectors = vectorizer.fit_transform([item[0] for item in chunked_sentences])
    except ValueError:
        # Handle case with too few sentences or no features, which can occur when document is empty or consists of only stop words
        # Fallback: take first sentences up to token limit
        selected_sentences = []
        total_tokens = 0
        
        for sentence, encoded in chunked_sentences:  # Use chunked_sentences for consistency
            sentence_tokens = len(encoded)
            if total_tokens + sentence_tokens <= max_tokens:
                selected_sentences.append((sentence, encoded))
                total_tokens += sentence_tokens
            else:
                break
        
        if output == 'tokens':
            return [item[1] for item in selected_sentences]
        elif output == 'text':
            return [item[0] for item in selected_sentences]
        else:
            warnings.warn('Invalid output. Defaulting to text.')
            return [item[0] for item in selected_sentences]
    
    if pooling == 'max':
        doc_centroid = np.max(sentence_vectors.toarray(), axis=0)
    elif pooling == 'sum':
        doc_centroid = np.sum(sentence_vectors.toarray(), axis=0)
    else:
        warnings.warn('Invalid pooling method. Defaulting to max pooling.')
        doc_centroid = np.max(sentence_vectors.toarray(), axis=0)

    mmr_selection = mmr(sentence_vectors.toarray(), doc_centroid, max_tokens=max_tokens, chunked_sentences=chunked_sentences, lambda_param=mmr_lambda)
    
    if output == 'tokens':
        return [selected[1] for selected in mmr_selection]
    elif output == 'text':
        return [selected[0] for selected in mmr_selection]
    else:
        warnings.warn('Invalid output. Defaulting to text.')
        return [selected[0] for selected in mmr_selection]

In [7]:
def transformers_consolidation(text: str,
                               pooling: str = 'mean',
                               model_name: str = 'sentence-transformers/all-MiniLM-L6-v2',
                               output: str = 'text',
                               chunking_method: str = 'sentence',
                               max_tokens: int = None,
                              mmr_lambda: float = 0.5) -> list:
    """
    Consolidate text by selecting sentences using embeddings and MMR algorithm.

    Embeds all sentences, computes a document centroid using specified chunking method 
    and pooling strategy, then applies Maximal Marginal Relevance (MMR) to select 
    sentences that balance similarity to the centroid with diversity among selections.
    
    Args:
        text (str): Input text to consolidate
        pooling (str): Pooling method for document centroid. Options: 'mean' (implemented), 
            'len_weighted', 'semantic_centrality' (not implemented). Defaults to 'mean'.
        model_name (str): SentenceTransformer model name for embeddings. 
            Defaults to 'sentence-transformers/all-MiniLM-L6-v2'.
        output (str): Output format - 'tokens' returns token lists, 'text' returns 
            strings. Defaults to 'text'.
        chunking_method (str): Method for computing document centroid:
            - 'sentence': Use sentence embeddings for centroid calculation
            - 'chunks': Use fixed-size chunk embeddings for centroid calculation
            Defaults to 'sentence'.
        max_tokens (int | None): Maximum tokens in output. If None, uses model's 
            maximum length. Defaults to None.
        mmr_lambda (float): MMR relevance vs diversity trade-off parameter 
            (1.0=pure relevance, 0.0=pure diversity). Defaults to 0.5.
       
    Returns:
        list[list[int]] | list[str]: Selected sentences as token lists or text strings 
            based on output parameter, ordered by MMR selection algorithm
            
    Raises:
        ValueError: If no sentences found in input text
    """
    # Initialize embedder
    embedder = SentenceTransformer(model_name)
    tokenizer = embedder.tokenizer
        
    # Auto-detect model's token limit
    if max_tokens is None:
        max_tokens = get_model_max_length(model_name)[0]
    
    # Split text into sentences (NB: does not split along newlines)
    sentences = sent_tokenize(text)
    if not sentences:
        raise ValueError("No sentences found in text.")
    encoded_sentences = [(sentence, tokenizer.encode(sentence)) for sentence in sentences]

    # Check if consolidation is needed
    text_tokens = sum(len(item[1]) for item in encoded_sentences)
    if text_tokens <= max_tokens: # No consolidation needed
        if output == 'tokens':
            return [item[1] for item in encoded_sentences]
        elif output == 'text':
            return [item[0] for item in encoded_sentences]
        else:
            warnings.warn('Invalid output. Defaulting to tokens.')
            return [item[1] for item in encoded_sentences]

    # Break apart sentences that are longer than max_tokens
    chunked_sentences = []
    for sentence, encoded in encoded_sentences:
        if len(encoded) <= max_tokens:            
            chunked_sentences.append((sentence, encoded))
        else:
            # TODO this approach is inefficient, but works for now
            for i in range(0, len(encoded), max_tokens):
                segment_text = tokenizer.decode(encoded[i:i + max_tokens], skip_special_tokens=True, clean_up_tokenization_spaces=True)
                segment_tokens = encoded[i:i + max_tokens]
                chunked_sentences.append((segment_text, segment_tokens))

    # Always embed sentences for selection
    sentence_embeddings = embedder.encode([item[0] for item in chunked_sentences], normalize_embeddings=False)
    
    # Embed and pool using pooling method of choice
    if chunking_method == 'sentence':        
        centroid_embeddings = sentence_embeddings  # Use sentence embeddings for centroid
    elif chunking_method == 'chunks':
        chunks = fixed_size_chunking(text, model_name=model_name)
        centroid_embeddings = embedder.encode(chunks, normalize_embeddings=False)  # Use chunk embeddings for centroid
    else:
        warnings.warn('Invalid chunking method. Defaulting to sentence chunking.')
        centroid_embeddings = sentence_embeddings

    if pooling == 'mean':
        doc_centroid = mean_pooling(centroid_embeddings)
    elif pooling == 'len_weighted':
        doc_centroid = len_weighted_mean_pooling(centroid_embeddings)
    elif pooling == 'semantic_centrality':
        doc_centroid = semantic_centrality_weighting(centroid_embeddings)
    else:
        warnings.warn('Invalid pooling method. Defaulting to mean pooling.')
        doc_centroid = mean_pooling(centroid_embeddings)
    
    mmr_selection = mmr(sentence_embeddings, doc_centroid, max_tokens=max_tokens, chunked_sentences=chunked_sentences, lambda_param=mmr_lambda)
    
    if output == 'tokens':
        return [selected[1] for selected in mmr_selection]
    elif output == 'text':
        return [selected[0] for selected in mmr_selection]
    else:
        warnings.warn('Invalid output. Defaulting to text.')
        return [selected[0] for selected in mmr_selection]

In [8]:
def summarization_pipeline_consolidation(text: str, model: str = 'sshleifer/distilbart-cnn-12-6', summary_size: int = None) -> str:
    """
    Consolidate text using iterative summarization until target size is reached.

    Chunks the input text and summarizes each chunk, then iteratively re-chunks 
    and re-summarizes the combined summaries until the result fits within the 
    target token limit. Uses a chunk ratio of 0.5 times the model's token limit 
    for processing chunks.
    
    Args:
        text (str): Input text to consolidate
        model (str): HuggingFace summarization model name. 
            Defaults to 'facebook/bart-large-cnn'.
        chunking_method (str): Method for chunking text. Options: 'fixed_size' 
            (implemented), 'paragraph', 'semantic' (may not be implemented). 
            Defaults to 'fixed_size'.
        summary_size (int | None): Target summary size in tokens. If None, 
            uses model's maximum length. Defaults to None.
       
    Returns:
        str: Consolidated summary text within the target token limit
    """
    summarizer = pipeline('summarization', model=model)
    tokenizer = AutoTokenizer.from_pretrained(model)
    model_token_limit = get_model_max_length(model)[0] 

    if summary_size is None:
        summary_size = model_token_limit

    if len(tokenizer.encode(text, truncation=False)) <= summary_size: # No consolidation needed
        return text

    chunk_ratio = 0.6
    doc_size = int(model_token_limit * chunk_ratio)
    docs = fixed_size_chunking(text, model_name=model, doc_size=doc_size)

    # directly summarize text without iterating, if possible
    if (len(docs) == 1) and (summary_size <= doc_size):
        doc_tokens = len(tokenizer.encode(docs[0]))
        min_length = min(math.ceil(doc_tokens * 0.3), math.ceil(summary_size * 0.6))
        
        return summarizer(
                docs[0],
                max_new_tokens=summary_size,
                max_length=summary_size,
                min_length=min_length,
                repetition_penalty=2.0,
                do_sample=False,
                num_beams=4,
                early_stopping=True
            )[0]['summary_text']

    while(True):   
        summaries = ''
        for doc in docs:
            doc_tokens = len(tokenizer.encode(doc))

            max_length = math.ceil(doc_tokens * 0.9)                                    
            min_length = math.ceil(doc_tokens * 0.3)

            # For debugging
            # print(f"About to summarize chunk with {doc_tokens} tokens. Limit: {summary_size}. max_length: {max_length}")
            # if doc_tokens > summary_size:
            #     print(f"WARNING: Chunk exceeds model limit!")
            
            summary = summarizer(
                doc,
                max_new_tokens=max_length,
                max_length=max_length,
                min_length=min_length,
                repetition_penalty=2.0,
                do_sample=False,
                num_beams=4,
                early_stopping=True
            )[0]['summary_text']
    
            summaries += summary + ' '

        summaries_tokenized = tokenizer.encode(summaries)

        # for debugging
        # print(len(summaries_tokenized))
        # TODO need a failsafe infinite loops. Maybe if len(summaries_tokenized) doesn't change, then break   
        if len(summaries_tokenized) <= summary_size:
            break
            
        docs = fixed_size_chunking(summaries, model_name=model, doc_size=int(model_token_limit * chunk_ratio))
    
    return summaries

In [9]:
def printing(text: str) -> None:
    print('*' * len('*** TF-IDF Consolidation ***'))
    print('*** TF-IDF Consolidation ***')
    print('*' * len('*** TF-IDF Consolidation ***'))
    print('Max pooling:')
    print('-' * len('Max pooling:'))
    print(' '.join(tfidf_consolidation(text, pooling='max')))
    print('=' * 60)
    print('Sum pooling:')
    print('-' * len('Sum pooling:'))
    print(' '.join(tfidf_consolidation(text, pooling='sum')))

    print()
    
    print('*' * len('*** Transformers Consolidation ***'))
    print('*** Transformers Consolidation ***')
    print('*' * len('*** Transformers Consolidation ***'))
    print('Sentence pooling:')
    print('-' * len('Sentence pooling:'))
    print(' '.join(transformers_consolidation(text, chunking_method='sentence')))
    print('=' * 60)
    print('Chunk pooling:')
    print('-' * len('Chunk pooling:'))
    print(' '.join(transformers_consolidation(text, chunking_method='chunks')))

# Text Parsing

In [10]:
king_james_bible = epub.read_epub('KJV.epub')

In [11]:
bible = {}
current_book = None

for item in king_james_bible.get_items():
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
        soup = BeautifulSoup(item.get_content(), 'html.parser').find('body')
        
        # book and chapter, e.g. Genesis 1
        title = soup.find('h2').get_text().strip()
        
        # remove book, chapter, and verse numbers from body text
        soup.find('h2').decompose()
        for span in soup.find_all('span'):
            span.decompose()
        
        # clean up whitespace and replace carriage returns with newlines
        body = soup.get_text().strip()
        body = body.replace('\r', '\n')

        # separate book from chapters
        if re.search(r'\d+$', title):
            if current_book:
                # Extract just the chapter number/name
                chapter_key = title.replace(current_book, '').strip()
                bible[current_book][int(chapter_key)] = body
        else:
            current_book = title
            bible[current_book] = {}

# Whole Bible

In [12]:
old_testament_books = [
    'Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy', 'Joshua', 'Judges', 'Ruth', '1 Samuel', '2 Samuel',
    '1 Kings', '2 Kings', '1 Chronicles', '2 Chronicles', 'Ezra', 'Nehemiah', 'Esther', 'Job', 'Psalms', 'Proverbs',
    'Ecclesiastes', 'Song of Solomon', 'Isaiah', 'Jeremiah', 'Lamentations', 'Ezekiel', 'Daniel', 'Hosea', 'Joel',
    'Amos', 'Obadiah', 'Jonah', 'Micah', 'Nahum', 'Habakkuk', 'Zephaniah', 'Haggai', 'Zechariah', 'Malachi'
]

new_testament_books = [
    'Matthew', 'Mark', 'Luke', 'John', 'Acts', 'Romans', '1 Corinthians', '2 Corinthians', 'Galatians',
    'Ephesians', 'Philippians', 'Colossians', '1 Thessalonians', '2 Thessalonians', '1 Timothy', '2 Timothy',
    'Titus', 'Philemon', 'Hebrews', 'James', '1 Peter', '2 Peter', '1 John', '2 John', '3 John', 'Jude',
    'Revelation'
]

all_books = old_testament_books + new_testament_books

bible_text = '\n'.join(get_book_text(book, bible) for book in all_books)
old_testament = '\n'.join(get_book_text(book, bible) for book in old_testament_books)
new_testament = '\n'.join(get_book_text(book, bible) for book in new_testament_books)

In [161]:
printing(bible_text)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
behold, thou hast made the heaven and the earth by thy great power and stretched out arm, and there is nothing too hard for thee:
  Thou shewest lovingkindness unto thousands, and recompensest the iniquity of the fathers into the bosom of their children after them: the Great, the Mighty God, the LORD of hosts, is his name,
  Great in counsel, and mighty in work: for thine eyes are open upon all the ways of the sons of men: to give every one according to his ways, and according to the fruit of his doings:
  Which hast set signs and wonders in the land of Egypt, even unto this day, and in Israel, and among other men; and hast made thee a name, as at this day;
  And hast brought forth thy people Israel out of the land of Egypt with signs, and with wonders, and with a strong hand, and with a stretched out arm, and with great terror;
  And hast given them this land, which thou di

In [13]:
summarization_pipeline_consolidation(bible_text)

C:\MyStuff\Applications\Productivity\Programming\Python\jupyter-env-313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lydp7\.cache\huggingface\hub\models--sshleifer--distilbart-cnn-12-6. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


' Noah begat three sons, Shem, Ham, and Japheth:\xa0and Noah was five hundred years old when the flood of waters was upon the earth . Noah went into the ark with his sons, his wife, and his sons\' wives with him . Afterward he vowed a vow, saying, \'If God will be with me,  then shall the LORD be my God\' Joseph was brought down to Egypt; and Potiphar, an officer of Pharaoh, captain of the guard, an Egyptian, bought him of the hands of the Ishmeelites . He shaved himself, and changed his raiment, and came in unto Pharaoh . And, behold, seven thin ears of corn came up upon one stalk, rank and good, and devoured the seven rank and full ears . So Joseph said, "God forbid that I should  "To obey is better than sacrifice, and to hearken than the fat of rams," says Abraham Abrahams son-in-law Sarah Abrahams daughter-to-son-of-law . Delilah made him shave off seven locks of his head, and she began to afflict him daily with her words . One of the sons of Ahimlech named Abiathar escaped, but Da

In [13]:
# from https://gist.github.com/sebleier/554280?permalink_comment_id=3813988#gistcomment-3813988 
# english stopwords list wasn't cutting it for KJV
stopwords_list = requests.get("https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt").content
stopwords = list(set(stopwords_list.decode().splitlines())) + ['thy', 'thee', 'thine', 'shalt', 'ye', 'hath', 'hast', 'saith', 'doeth', 'art']

vectorizer = TfidfVectorizer(ngram_range=(1, 3), stop_words=stopwords)
    
testament_vectors = vectorizer.fit_transform([old_testament, new_testament])
feature_names = vectorizer.get_feature_names_out()

# Find top 5 words for each book
for i, testament in enumerate(['Old Testament', 'New Testament']):
    # TF-IDF scores for current book
    testament_tfidf = testament_vectors[i].toarray().flatten()    
    # Indices of top 5 scores
    top_indices = testament_tfidf.argsort()[-5:][::-1]    
    # Words and their scores
    top_words = [(feature_names[idx], testament_tfidf[idx]) for idx in top_indices]
    
    print(f"\n{testament}:")
    for word, score in top_words:
        print(f"  {word}: {score:.4f}")

C:\MyStuff\Applications\Productivity\Programming\Python\jupyter-env-313\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['articl', 'mon'] not in stop_words.
  warnings.warn(



Old Testament:
  lord: 0.6175
  god: 0.2644
  israel: 0.2132
  king: 0.2090
  son: 0.1681

New Testament:
  jesus: 0.4256
  god: 0.4227
  man: 0.2797
  christ: 0.2472
  things: 0.2246


In [30]:
# from https://gist.github.com/sebleier/554280?permalink_comment_id=3813988#gistcomment-3813988 
# english stopwords list wasn't cutting it for KJV
stopwords_list = requests.get("https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt").content
stopwords = list(set(stopwords_list.decode().splitlines())) + ['thy', 'thee', 'thine', 'shalt', 'ye', 'hath', 'hast', 'saith', 'doeth', 'art']

vectorizer = TfidfVectorizer(ngram_range=(1, 3), stop_words=stopwords)
    
book_vectors = vectorizer.fit_transform([get_book_text(book, bible) for book in all_books])
feature_names = vectorizer.get_feature_names_out()

# Find top 5 words for each book
for i, book in enumerate(all_books):
    # TF-IDF scores for current book
    book_tfidf = book_vectors[i].toarray().flatten()    
    # Indices of top 5 scores
    top_indices = book_tfidf.argsort()[-5:][::-1]    
    # Words and their scores
    top_words = [(feature_names[idx], book_tfidf[idx]) for idx in top_indices]
    
    print(f"\n{book}:")
    for word, score in top_words:
        print(f"  {word}: {score:.4f}")


Genesis:
  jacob: 0.2118
  joseph: 0.2064
  land: 0.1669
  abraham: 0.1628
  father: 0.1561

Exodus:
  moses: 0.3317
  lord: 0.2877
  aaron: 0.1809
  pharaoh: 0.1750
  israel: 0.1482

Leviticus:
  offering: 0.3312
  lord: 0.2442
  priest: 0.2329
  unclean: 0.1550
  aaron: 0.1374

Numbers:
  lord: 0.2469
  offering: 0.2439
  moses: 0.2308
  israel: 0.1785
  children: 0.1733

Deuteronomy:
  lord: 0.4679
  lord god: 0.3581
  god: 0.2896
  land: 0.2021
  people: 0.0960

Joshua:
  joshua: 0.4302
  lord: 0.2241
  children: 0.2035
  israel: 0.1898
  suburbs: 0.1654

Judges:
  israel: 0.2737
  lord: 0.2176
  gideon: 0.1963
  samson: 0.1741
  children: 0.1698

Ruth:
  naomi: 0.3761
  boaz: 0.2854
  ruth: 0.1956
  kinsman: 0.1855
  mother law: 0.1355

1 Samuel:
  saul: 0.5958
  david: 0.3444
  samuel: 0.2628
  lord: 0.2421
  philistines: 0.1741

2 Samuel:
  david: 0.4374
  king: 0.3305
  absalom: 0.2898
  joab: 0.2627
  lord: 0.1758

1 Kings:
  king: 0.3290
  solomon: 0.3193
  lord: 0.2607
  is

# Old Testament

## Full Old Testament

In [18]:
printing(old_testament)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
behold, thou hast made the heaven and the earth by thy great power and stretched out arm, and there is nothing too hard for thee:
  Thou shewest lovingkindness unto thousands, and recompensest the iniquity of the fathers into the bosom of their children after them: the Great, the Mighty God, the LORD of hosts, is his name,
  Great in counsel, and mighty in work: for thine eyes are open upon all the ways of the sons of men: to give every one according to his ways, and according to the fruit of his doings:
  Which hast set signs and wonders in the land of Egypt, even unto this day, and in Israel, and among other men; and hast made thee a name, as at this day;
  And hast brought forth thy people Israel out of the land of Egypt with signs, and with wonders, and with a strong hand, and with a stretched out arm, and with great terror;
  And hast given them this land, which thou di

In [47]:
summarization_pipeline_consolidation(old_testament)

' Noah begat three sons, Shem, Ham, and Japheth:\xa0and Noah was five hundred years old when the flood of waters was upon the earth . Noah went into the ark with his sons, his wife, and his sons\' wives with him . Afterward he vowed a vow, saying, \'If God will be with me,  then shall the LORD be my God\' Joseph was brought down to Egypt; and Potiphar, an officer of Pharaoh, captain of the guard, an Egyptian, bought him of the hands of the Ishmeelites . He shaved himself, and changed his raiment, and came in unto Pharaoh . And, behold, seven thin ears of corn came up upon one stalk, rank and good, and devoured the seven rank and full ears . So Joseph said, "God forbid that I should  "To obey is better than sacrifice, and to hearken than the fat of rams," says Abraham Abrahams son-in-law Sarah Abrahams daughter-to-son-of-law . Delilah made him shave off seven locks of his head, and she began to afflict him daily with her words . One of the sons of Ahimlech named Abiathar escaped, but Da

## Genesis

In [15]:
genesis = get_book_text('Genesis', bible)

In [16]:
printing(genesis)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And the LORD appeared unto him in the plains of Mamre: and he sat in the tent door in the heat of the day;
  And he lift up his eyes and looked, and, lo, three men stood by him: and when he saw them, he ran to meet them from the tent door, and bowed himself toward the ground,
  And said, My Lord, if now I have found favour in thy sight, pass not away, I pray thee, from thy servant:
  Let a little water, I pray you, be fetched, and wash your feet, and rest yourselves under the tree:
  And I will fetch a morsel of bread, and comfort ye your hearts; after that ye shall pass on: for therefore are ye come to your servant. And Abram took Sarai his wife, and Lot his brother's son, and all their substance that they had gathered, and the souls that they had gotten in Haran; and they went forth to go into the land of Canaan; and into the land of Canaan they came. Now the sons of Jacob

In [50]:
summarization_pipeline_consolidation(genesis)

' Adam said, This is now bone of my bones, and flesh of my flesh: she shall be called Woman, because she was taken out of Man . So God created man in his own image, in the image of God created he created he him; male and female created he them . Noah begat three sons, Shem, Ham, and Japheth:\xa0and Noah was five hundred years old when the flood of waters was upon the earth . Noah went into the ark with his sons, his wife, and his sons\' wives with him . After the end of the hundred-and-fifty days the waters were abated. And all flesh died both of fowl, and of cattle, and beast, and each man . All in whose nostrils was the breath of life  Abraham said to his eldest servant of his house, that he would not take a wife to my son of the daughters of the Canaanites, among whom I dwell:  But thou shalt go to my kindred, and take a marriage unto my child Isaac . Abraham\'s servant took ten camels of his master, and departed . He arose, and went to Mesopotamia, unto the city of Nahor . Esau too

## Exodus

In [19]:
exodus = get_book_text('Exodus', bible)

In [20]:
printing(exodus)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And they brought the tabernacle unto Moses, the tent, and all his furniture, his taches, his boards, his bars, and his pillars, and his sockets,
  And the covering of rams' skins dyed red, and the covering of badgers' skins, and the vail of the covering,
  The ark of the testimony, and the staves thereof, and the mercy seat,
  The table, and all the vessels thereof, and the shewbread,
  The pure candlestick, with the lamps thereof, even with the lamps to be set in order, and all the vessels thereof, and the oil for light,
  And the golden altar, and the anointing oil, and the sweet incense, and the hanging for the tabernacle door,
  The brasen altar, and his grate of brass, his staves, and all his vessels, the laver and his foot,
  The hangings of the court, his pillars, and his sockets, and the hanging for the court gate, his cords, and his pins, and all the vessels of the 

In [53]:
summarization_pipeline_consolidation(exodus)

" The children of Israel journeyed from Rameses to Succoth, about six hundred thousand on foot that were men, beside children . And they baked unleavened cakes of the dough which they brought forth out of Egypt, for it was not leavened . A foreigner and an hired servant shall not eat thereof . In one house shall not a dog move his tongue, nor shall ye break a bone thereof . And thus shall ye eat it; with your loins girded, your shoes on your feet, and your staff in your hand; and ye shall eat it in haste: it is the LORD'S passover . Miriam the prophetess, the sister of Aaron, took a timbrel in her hand; all the women went out after her . Jethro said, 'I am a jealous God, visiting the iniquity of the fathers upon the children  Aaron, Nadab and Abihu, Eleazar and Ithamar, Aaron's sons, shall make holy garments for Aaron thy brother . And they shall make the ephod of gold, of blue, and purple, and scarlet, and fine twined linen, with cunning work . Two onyx stones, and grave on them the n

## Leviticus

In [21]:
leviticus = get_book_text('Leviticus', bible)

In [22]:
printing(leviticus)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And the LORD spake unto Moses, saying,
  If a soul commit a trespass, and sin through ignorance, in the holy things of the LORD; then he shall bring for his trespass unto the LORD a ram without blemish out of the flocks, with thy estimation by shekels of silver, after the shekel of the sanctuary, for a trespass offering:
  And he shall make amends for the harm that he hath done in the holy thing, and shall add the fifth part thereto, and give it unto the priest: and the priest shall make an atonement for him with the ram of the trespass offering, and it shall be forgiven him. This is the law for all manner of plague of leprosy, and scall,
  And for the leprosy of a garment, and of a house,
  And for a rising, and for a scab, and for a bright spot:
  To teach when it is unclean, and when it is clean: this is the law of leprosy. And whatsoever man there be of the children of I

In [56]:
summarization_pipeline_consolidation(leviticus)

" Moses spoke to the children of Israel, and say: If any man of you bring an offering unto the LORD, ye shall bring your offering of the cattle, even of the herd, and of the flock . And if his offering be a burnt sacrifice, let him offer a male without blemish: he shall offer it of his own voluntary will at the door of the tabernacle of the congregation before the LORD . But his inwards and his legs shall he wash in water:   When any will offer a meat offering to the Lord, his offering shall be of fine flour; and he shall pour oil upon it, and put frankincense thereon:  And he shall bring it to Aaron's sons the priests . And the priest shall burn it upon the altar, upon the wood that is upon the fire which is on the altar .  The flesh that toucheth any unclean thing shall not be eaten; it shall be burnt with fire . But the soul that eateth of  peace offerings, having his uncleanness upon him, even that soul shall be cut off from his people . Aaron slew the calf of the sin offering, whi

## Numbers

In [23]:
numbers = get_book_text('Numbers', bible)

In [24]:
printing(numbers)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And Moses sent messengers from Kadesh unto the king of Edom, Thus saith thy brother Israel, Thou knowest all the travail that hath befallen us:
  How our fathers went down into Egypt, and we have dwelt in Egypt a long time; and the Egyptians vexed us, and our fathers:
  And when we cried unto the LORD, he heard our voice, and sent an angel, and hath brought us forth out of Egypt: and, behold, we are in Kadesh, a city in the uttermost of thy border:
  Let us pass, I pray thee, through thy country: we will not pass through the fields, or through the vineyards, neither will we drink of the water of the wells: we will go by the king's high way, we will not turn to the right hand nor to the left, until we have passed thy borders. On the second day Nethaneel the son of Zuar, prince of Issachar, did offer:
  He offered for his offering one silver charger, the weight whereof was an 

In [59]:
summarization_pipeline_consolidation(numbers)

" The children of Reuben, Israel's eldest son, by their generations, after their families, by the house of their fathers, with the number of their names . From twenty years old and upward, all that are able to go forth to war in Israel: thou and Aaron shall number them by their armies . With you there shall be a man of every tribe; every one head of the house  of his fathers . This also are the generations of Aaron and Moses in the day that the LORD spake with Moses in mount Sinai . Those that were numbered of the camps throughout their hosts were six hundred thousand and three thousand and five hundred and fifty . But the Levite were not numbered among the children of Israel; as the LORD commanded Moses .  Moses commanded the children of Israel to make fringes in the borders of their garments throughout their generations . If any soul sin through ignorance, then he shall bring a goat of the first year for a sin offering . Two hundred and fifty men that offered incense were consumed by

## Deuteronomy

In [25]:
deuteronomy = get_book_text('Deuteronomy', bible)

In [26]:
printing(deuteronomy)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Neither with you only do I make this covenant and this oath;
  But with him that standeth here with us this day before the LORD our God, and also with him that is not here with us this day:
  (For ye know how we have dwelt in the land of Egypt; and how we came through the nations which ye passed by;
  And ye have seen their abominations, and their idols, wood and stone, silver and gold, which were among them:)
  Lest there should be among you man, or woman, or family, or tribe, whose heart turneth away this day from the LORD our God, to go and serve the gods of these nations; lest there should be among you a root that beareth gall and wormwood;
  And it come to pass, when he heareth the words of this curse, that he bless himself in his heart, saying, I shall have peace, though I walk in the imagination of mine heart, to add drunkenness to thirst:
  The LORD will not spare hi

In [62]:
summarization_pipeline_consolidation(deuteronomy)

" Moses spake unto all Israel on this side Jordan in the wilderness, in the plain over against the Red sea . Behold, I have set the land before you  The Amorites came to Kadeshbarnea, a mountain of the Amorite . And I commanded you at that time all the things which ye should do . Ye shall not respect persons in judgment; but ye shall hear the small as well as the great . For the judgment is God's: and the cause that is too hard for you, bring it unto me, and I will hear it . Joshua the son of Nun, which standeth before thee, he shall go in thither: encourage him: for he shall cause Israel to inherit it . This day will I begin to put the dread of thee and the fear of thee upon the nations that are under the whole heaven,  Aaron made an ark of shittim wood, and hewed two tables of stone like unto the first . The children of Israel took their journey from Beeroth of the children of Jaakan to Mosera . Aaron died, and there he was buried; and Eleazar his son ministered in the priest's offic

## Joshua

In [27]:
joshua = get_book_text('Joshua', bible)

In [28]:
printing(joshua)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Now therefore divide this land for an inheritance unto the nine tribes, and the half tribe of Manasseh,
  With whom the Reubenites and the Gadites have received their inheritance, which Moses gave them, beyond Jordan eastward, even as Moses the servant of the LORD gave them;
  From Aroer, that is upon the bank of the river Arnon, and the city that is in the midst of the river, and all the plain of Medeba unto Dibon;
  And all the cities of Sihon king of the Amorites, which reigned in Heshbon, unto the border of the children of Ammon;
  And Gilead, and the border of the Geshurites and Maachathites, and all mount Hermon, and all Bashan unto Salcah;
  All the kingdom of Og in Bashan, which reigned in Ashtaroth and in Edrei, who remained of the remnant of the giants: for these did Moses smite, and cast them out. And it came to pass after three days, that the officers went throug

In [65]:
summarization_pipeline_consolidation(joshua)

' Joshua the son of Nun, Moses\' minister, said, "Now therefore arise, go over this Jordan, thou, and all this people, unto the land which I do give to them, even to the children of Israel" Joshua commanded the officers of the people, saying, Pass through the host, and command the people . Rahab the harlot hid messengers, which Joshua sent to spy out Jericho, and she hid them . But she said, I know that the LORD  Moses spake unto Joshua, saying , Command the priests that bear the ark of the testimony, that they come up out of Jordan . And it came to pass, when all the people were clean passed over, that the arks of the LORD passed over . The children of Reuben and Gad, and half the tribe of Manasseh, passed over armed  In the morning therefore ye shall be brought according to your tribes: and it shall be, that the tribe which the LORD taketh shall come according to the families thereof . So Joshua rose up early in the morning, and brought Israel by their tribes; and the tribe of Judah 

## Judges

In [29]:
judges = get_book_text('Judges', bible)

In [151]:
printing(judges)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And it came to pass, when the children of Israel cried unto the LORD because of the Midianites,
  That the LORD sent a prophet unto the children of Israel, which said unto them, Thus saith the LORD God of Israel, I brought you up from Egypt, and brought you forth out of the house of bondage;
  And I delivered you out of the hand of the Egyptians, and out of the hand of all that oppressed you, and drave them out from before you, and gave you their land;
  And I said unto you, I am the LORD your God; fear not the gods of the Amorites, in whose land ye dwell: but ye have not obeyed my voice. Now therefore up by night, thou and the people that is with thee, and lie in wait in the field:
  And it shall be, that in the morning, as soon as the sun is up, thou shalt rise early, and set upon the city: and, behold, when he and the people that is with him come out against thee, then ma

In [68]:
summarization_pipeline_consolidation(judges)

' The children of Israel asked the LORD, saying, Who shall go up for us against the Canaanites first, to fight against them? And the LORD said, Judah will go up: behold, I have delivered the land into his hand . And Judah went up; and the LORD delivered the Canaanite and the Perizzites into their hand . But Adonibezek fled; and they pursued after him, and cut off his thumbs and his great toes . He said, Threescore and ten kings, having their thumbs and their great toes cut off, gathered their meat under my table: as I have done, so God hath requited me . And they brought him to Jerusalem, and there he died . They gave Hebron unto Caleb, as Moses said: and he expelled the three sons of Anak .  Gideon put a fleece of wool in the floor, and wringed the dew out of the fleece, a bowl full of water . The men of the city said to Joash, Bring out thy son, that he may die: because he has cast down the altar of Baal . And Joash said to all that stood against him, Will ye plead for Baal? will ye 

## Ruth

In [31]:
ruth = get_book_text('Ruth', bible)

In [32]:
printing(ruth)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And the servant that was set over the reapers answered and said, It is the Moabitish damsel that came back with Naomi out of the country of Moab:
  And she said, I pray you, let me glean and gather after the reapers among the sheaves: so she came, and hath continued even from the morning until now, that she tarried a little in the house. And now, my daughter, fear not; I will do to thee all that thou requirest: for all the city of my people doth know that thou art a virtuous woman. Moreover Ruth the Moabitess, the wife of Mahlon, have I purchased to be my wife, to raise up the name of the dead upon his inheritance, that the name of the dead be not cut off from among his brethren, and from the gate of his place: ye are witnesses this day. And she went down unto the floor, and did according to all that her mother in law bade her. And he turned aside, and sat down. Now this was

In [71]:
summarization_pipeline_consolidation(ruth)

" A certain man of Bethlehemjudah went to sojourn in the country of Moab, he and his wife, and his two sons . Elimelech Naomi's husband died; and she was left, and her two sons Mahlon and Chilion . She arose with her daughters in law, that she might return to the land of Judah . And Naomi said, Turn again, my daughters: why will ye go with me? Are there yet any more sons in my womb, that they may be your husbands? For it grieveth me much for your sakes that the hand of the LORD is gone out against me . But Orpah kissed her mother in law; but Ruth clave unto her. And she said, Behold, thy sister in law is gone back unto her people, and unto her gods: return  Naomi had a kinsman of her husband's, a mighty man of wealth, of the family of Elimelech; and his name was Boaz . Naomi returned, and Ruth the Moabitess, her daughter in law, with her, which returned out of the country of Moab . They came to Bethlehem in the beginning of barley harvest . And Ruth said, Intreat me not to leave thee, 

## 1 Samuel

In [33]:
samuel1 = get_book_text('1 Samuel', bible)

In [34]:
printing(samuel1)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And Jonathan spake good of David unto Saul his father, and said unto him, Let not the king sin against his servant, against David; because he hath not sinned against thee, and because his works have been to thee-ward very good:
  For he did put his life in his hand, and slew the Philistine, and the LORD wrought a great salvation for all Israel: thou sawest it, and didst rejoice: wherefore then wilt thou sin against innocent blood, to slay David without a cause? And they answered them, and said, He is; behold, he is before you: make haste now, for he came to day to the city; for there is a sacrifice of the people to day in the high place:
  As soon as ye be come into the city, ye shall straightway find him, before he go up to the high place to eat: for the people will not eat until he come, because he doth bless the sacrifice; and afterwards they eat that be bidden. And when 

In [74]:
summarization_pipeline_consolidation(samuel1)

" Hannah was in bitterness of soul, and prayed unto the LORD, and wept sore . She vowed a vow, and said, O LORD of hosts, if thou wilt indeed look on the affliction of thine handmaid, and remember me, then I will give him all the days of his life, and there shall no razor come upon his head . But it came to pass, as she continued praying before the Lord, that Eli marked her mouth . And therefore I have sworn unto the house of Eli, that the iniquity of Eli's house shall not be purged with sacrifice nor offering for ever . Now Israel went out against the Philistines to battle, and pitched beside Ebenezer: and there was a very great slaughter; for there fell of Israel thirty thousand footmen . The bows of the mighty men are broken, and they  The children of Israel did put away Baalim and Ashtoth, and served the LORD only . But the Philistines drew near to battle against Israel: but the LORD thundered with a great thunder on that day, and discomfited them; and they were smitten before Isra

## 2 Samuel

In [35]:
samuel2 = get_book_text('2 Samuel', bible)

In [36]:
printing(samuel2)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And the woman answered and said, As thy soul liveth, my lord the king, none can turn to the right hand or to the left from ought that my lord the king hath spoken: for thy servant Joab, he bade me, and he put all these words in the mouth of thine handmaid:
  To fetch about this form of speech hath thy servant Joab done this thing: and my lord is wise, according to the wisdom of an angel of God, to know all things that are in the earth. Moreover Ahithophel said unto Absalom, Let me now choose out twelve thousand men, and I will arise and pursue after David this night:
  And I will come upon him while he is weary and weak handed, and will make him afraid: and all the people that are with him shall flee; and I will smite the king only:
  And I will bring back all the people unto thee: the man whom thou seekest is as if all returned: so all the people shall be in peace.
Sum pool

In [77]:
summarization_pipeline_consolidation(samuel2)

" David stood upon him, and slew him, because I was sure that he could not live after that he was fallen . Then David took hold on his clothes, and rent them; and likewise all the men that were with him mourned, and wept, and fasted until even . Joab smothered Abner when he was returned to Hebron, took him aside in the gate to speak with him quietly, and smote him there under the fifth rib, that he died . After David heard it, he said, I and my kingdom are guiltless before the LORD for ever from the blood of Abner . As soon as David had made an end of offering burnt offerings, he blessed the people in the name of the LORD of hosts . Michal Saul's daughter looked through a window, and saw king David leaping and dancing before the Lord  Nathan said to the king, Go, do all that is in thine heart; for the LORD is with thee . So all the people departed every one to his house . Then David returned to bless his household . He said, See now, I dwell in an house of cedar, but the ark of God dwe

## 1 Kings

In [37]:
kings1 = get_book_text('1 Kings', bible)

In [38]:
printing(kings1)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And it came to pass, as they sat at the table, that the word of the LORD came unto the prophet that brought him back:
  And he cried unto the man of God that came from Judah, saying, Thus saith the LORD, Forasmuch as thou hast disobeyed the mouth of the LORD, and hast not kept the commandment which the LORD thy God commanded thee,
  But camest back, and hast eaten bread and drunk water in the place, of the which the LORD did say to thee, Eat no bread, and drink no water; thy carcase shall not come unto the sepulchre of thy fathers. And Benhadad the king of Syria gathered all his host together: and there were thirty and two kings with him, and horses, and chariots: and he went up and besieged Samaria, and warred against it.
Sum pooling:
------------
And the king of Israel answered and said, My lord, O king, according to thy saying, I am thine, and all that I have. And she sai

In [80]:
summarization_pipeline_consolidation(kings1)

' Solomon\'s provision for one day was thirty measures of fine flour, and threescore measures of meal . The days that David reigned over Israel were forty years: seven years reigned he in Hebron, and thirty and three years in Jerusalem . Adonijah the son of Haggith exalted himself, saying, I will be king . He prepared chariots and horsemen, and fifty men to run before him . But Nathan the prophet, and Benaiah, and the mighty men, and Solomon his brother, he called not . Nathan spake unto Bathsheba the mother of Solomon, saying: Hast thou not heard that Adon Elijah doth reign, and David our lord knoweth it not? Then Bathsheaba bowed with her face to the earth, and said, Let my lord king David live for ever .  Solomon was building his own house 13 years, and he finished all his house . He overlaid the house within with pure gold; and so covered the altar which was of cedar . The height of the one cherubim was ten cubits, and so was it of the other . Both the cherubims were of one measure

## 2 Kings

In [39]:
kings2 = get_book_text('2 Kings', bible)

In [40]:
printing(kings2)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And Elisha the prophet called one of the children of the prophets, and said unto him, Gird up thy loins, and take this box of oil in thine hand, and go to Ramothgilead:
  And when thou comest thither, look out there Jehu the son of Jehoshaphat the son of Nimshi, and go in, and make him arise up from among his brethren, and carry him to an inner chamber;
  Then take the box of oil, and pour it on his head, and say, Thus saith the LORD, I have anointed thee king over Israel. And the captain of the guard took Seraiah the chief priest, and Zephaniah the second priest, and the three keepers of the door:
  And out of the city he took an officer that was set over the men of war, and five men of them that were in the king's presence, which were found in the city, and the principal scribe of the host, which mustered the people of the land, and threescore men of the people of the land

In [83]:
summarization_pipeline_consolidation(kings2)

" Ahaziah fell down through a lattice in his upper chamber that was in Samaria, and was sick . He sent messengers to enquire of Baalzebub the god of Ekron whether I shall recover of this disease . But the angel of the LORD said to Elijah the Tishbite, Arise, go up to meet the messengers of the king of Samaria . And he said to him, If I be a man of God, then let fire come down from heaven, and consume thee and thy fifty . And there came down fire from heaven and consumed him and his fifty . The waters were healed unto this day, according to the saying of Elishi which he spake . Naaman refused, saying: Are not Abana and Pharpar, rivers of Damascus, better than all the waters of Israel? may I not wash in them, and be  Naaman went out from his presence a leper as white as snow . An ass's head was sold for fourscore pieces of silver, and the fourth part of a cab of dove's dung for five pieces of gold . So the bands of Syria came no more into the land of Israel . Jehu wrote letters to the ru

## 1 Chronicles

In [41]:
chronicles1 = get_book_text('1 Chronicles', bible)

In [152]:
printing(chronicles1)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
So he left there before the ark of the covenant of the LORD Asaph and his brethren, to minister before the ark continually, as every day's work required:
  And Obededom with their brethren, threescore and eight; Obededom also the son of Jeduthun and Hosah to be porters:
  And Zadok the priest, and his brethren the priests, before the tabernacle of the LORD in the high place that was at Gibeon,
  To offer burnt offerings unto the LORD upon the altar of the burnt offering continually morning and evening, and to do according to all that is written in the law of the LORD, which he commanded Israel;
  And with them Heman and Jeduthun, and the rest that were chosen, who were expressed by name, to give thanks to the LORD, because his mercy endureth for ever;
  And with them Heman and Jeduthun with trumpets and cymbals for those that should make a sound, and with musical instruments

In [86]:
summarization_pipeline_consolidation(chronicles1)

" Adam, Sheth, Enosh, Kenan, Jered, Methuselah, Lamech, and Noah . Noah, Shem, Ham, and Japheth . The sons of Gomer; Ashchenaz, and Riphath, and Togarmah . And Cush begat Nimrod: he began to be mighty upon the earth . Mizraim begat Ludim, and Anamim,. and Lehabim,  Naphtuhim, (of whom came Philistines,) and Caphthorim . Shobal the fathers of Kirjathjearim had sons; Haroeh, and half of the Manahethites, the families of the scribes which dwelt at Jabez; the Tirathites . These are the Kenites that came of Hemath, the father  of Rechab .  These are the families of the Levites according to their fathers, David ministered before the dwelling place of the tabernacle of the congregation with singing, until Solomon had built the house of the LORD in Jerusalem: and then they waited on their office according to order . For example, Amaziah, Shimea, Malchiah, Adahiah, Adaiah, Zimmah, Zephaniah,  The sons of Aaron they gave the cities of Judah, namely, Hebron and Libnah with her suburbs . And Hilen

## 2 Chronicles

In [43]:
chronicles2 = get_book_text('2 Chronicles', bible)

In [44]:
printing(chronicles2)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And he stood before the altar of the LORD in the presence of all the congregation of Israel, and spread forth his hands:
  For Solomon had made a brasen scaffold, of five cubits long, and five cubits broad, and three cubits high, and had set it in the midst of the court: and upon it he stood, and kneeled down upon his knees before all the congregation of Israel, and spread forth his hands toward heaven,
  And said, O LORD God of Israel, there is no God like thee in the heaven, nor in the earth; which keepest covenant, and shewest mercy unto thy servants, that walk before thee with all their hearts:
  Thou which hast kept with thy servant David my father that which thou hast promised him; and spakest with thy mouth, and hast fulfilled it with thine hand, as it is this day. Moreover the LORD stirred up against Jehoram the spirit of the Philistines, and of the Arabians, that we

In [89]:
summarization_pipeline_consolidation(chronicles2)

" Solomon the son of David was strengthened in his kingdom, and the LORD his God was with him, and magnified him exceedingly . He overlaid it within with pure gold; and graved cherubims on the walls . And the greater house he cieled with fir tree, which he overlayed with fine gold, and set thereon palm trees and chains . He told out threescore and ten thousand men to bear burdens, and fourscore thousand to hew in the mountain, and three thousand and six hundred to oversee them . And, behold, I will give to thy servants, the hewers that cut timber, twenty thousand measures of beaten wheat, and twenty thousand baths of wine, and \xa0Solomon numbered all the strangers  Huram finished the work that he was to make for king Solomon for the house of God .  Solomon slept with his fathers, and he was buried in the city of David his father . Rehoboam his son reigned in his stead . But Abijah waxed mighty, and married 14 wives, and begat twenty and two sons, and sixteen daughters . He reigned ove

## Ezra

In [45]:
ezra = get_book_text('Ezra', bible)

In [46]:
printing(ezra)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Forasmuch as thou art sent of the king, and of his seven counsellors, to enquire concerning Judah and Jerusalem, according to the law of thy God which is in thine hand;
  And to carry the silver and gold, which the king and his counsellors have freely offered unto the God of Israel, whose habitation is in Jerusalem,
  And all the silver and gold that thou canst find in all the province of Babylon, with the freewill offering of the people, and of the priests, offering willingly for the house of their God which is in Jerusalem:
  That thou mayest buy speedily with this money bullocks, rams, lambs, with their meat offerings and their drink offerings, and offer them upon the altar of the house of your God which is in Jerusalem. Of the sons of Bani; Maadai, Amram, and Uel,
  Benaiah, Bedeiah, Chelluh,
  Vaniah, Meremoth, Eliashib,
  Mattaniah, Mattenai, and Jaasau,
  And Bani, an

In [92]:
summarization_pipeline_consolidation(ezra)

" The children of Shephatiah, three hundred seventy and two. Children of Pahathmoab, of the children of Jeshua and Joab, two thousand eight hundred hundred hundred . and twelve . The men of Elam, a thousand two hundred fifty and four. Men of Bethel and Ai, two hundred twenty and three. The sons of Zaccai, seven hundred and threescore. Beside their servants and their maids, there were among them two hundred singing men and singing women . These sought their register among those that were reckoned by genealogy, but they were not found: therefore were . therefore were they, as polluted to the priesthood. For more information, visit http://www.mailonline.co.uk/news/miracle-miracula-miracle\xa0and  Some of the chief of the fathers, when they came to the house of the LORD which is at Jerusalem, offered freely for it to be set up in his place:  They gave after their ability unto the treasure of the work threeescore and one thousand drams of gold, and five thousand pound of silver, and one hun

## Nehemiah

In [47]:
nehemiah = get_book_text('Nehemiah', bible)

In [48]:
printing(nehemiah)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
The chief of the people; Parosh, Pahathmoab, Elam, Zatthu, Bani,
  Bunni, Azgad, Bebai,
  Adonijah, Bigvai, Adin,
  Ater, Hizkijah, Azzur,
  Hodijah, Hashum, Bezai,
  Hariph, Anathoth, Nebai,
  Magpiash, Meshullam, Hezir,
  Meshezabeel, Zadok, Jaddua,
  Pelatiah, Hanan, Anaiah,
  Hoshea, Hananiah, Hashub,
  Hallohesh, Pileha, Shobek,
  Rehum, Hashabnah, Maaseiah,
  And Ahijah, Hanan, Anan,
  Malluch, Harim, Baanah. And it came to pass, when I heard these words, that I sat down and wept, and mourned certain days, and fasted, and prayed before the God of heaven,
  And said, I beseech thee, O LORD God of heaven, the great and terrible God, that keepeth covenant and mercy for them that love him and observe his commandments:
  Let thine ear now be attentive, and thine eyes open, that thou mayest hear the prayer of thy servant, which I pray before thee now, day and night, for the 

In [95]:
summarization_pipeline_consolidation(nehemiah)

' Nehemiah the son of Hachaliah said, "The wall of Jerusalem also is broken down, and the gates thereof are burned with fire" The remnant that are left of the captivity there in the province are in great affliction and reproach . Eliashib the high priest rose up with his brethren the priests, and they builded the sheep gate . They sanctified it, and set up the doors of it, unto the tower of Meah . But Sanballat the Horonite, Tobiah the servant, the Ammonite, and Geshem the Arabian, heard it and said, What is this thing that ye do? will ye rebel against the king? Then answered I them, The God of heaven, he will prosper us . And the rulers knew not whither I went, or what I did   Some of the chief of the fathers gave to the treasure a thousand drams of gold, fifty basons, five hundred and thirty priests\' garments . These sought their register among those that were reckoned by genealogy, but it was not found: therefore were they, as polluted, put from priesthood . Ezra the priest brought

## Esther

In [49]:
esther = get_book_text('Esther', bible)

In [50]:
printing(esther)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Again Esther spake unto Hatach, and gave him commandment unto Mordecai;
  All the king's servants, and the people of the king's provinces, do know, that whosoever, whether man or woman, shall come unto the king into the inner court, who is not called, there is one law of his to put him to death, except such to whom the king shall hold out the golden sceptre, that he may live: but I have not been called to come in unto the king these thirty days. For the Jews that were in Shushan gathered themselves together on the fourteenth day also of the month Adar, and slew three hundred men at Shushan; but on the prey they laid not their hand. And the king loved Esther above all the women, and she obtained grace and favour in his sight more than all the virgins; so that he set the royal crown upon her head, and made her queen instead of Vashti. And the thing pleased Haman; and he caused

In [98]:
summarization_pipeline_consolidation(esther)

' The queen Vashti refused to come at the king\'s commandment by his chamberlains . Memucan, the seven princes of Persia and Media, asked: What shall we do to the queen according to law? And Memucans answered: "Vashti has not done wrong to the king only, but also to all the princes, and to all people that are in all the provinces of the king ... for this deed of the queen shall come abroad unto all women, so that they shall despise their husbands in their eyes" After that she had been twelve months, according to the manner of the way the women, (for it is great,) all the wives shall give to their husbands honour, both to great and small  Esther had neither father nor mother, and the maid was fair and beautiful;  The king made a great feast unto all his princes and his servants, even Esther\'s feast . Then the king set the royal crown upon her head, and made her queen instead of Vashti . After these things did king Ahasuerus promote Haman the son of Hammedatha the Agagite, and advanced 

## Job

In [51]:
job = get_book_text('Job', bible)

In [52]:
printing(job)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Moreover Job continued his parable, and said,
  Oh that I were as in months past, as in the days when God preserved me;
  When his candle shined upon my head, and when by his light I walked through darkness;
  As I was in the days of my youth, when the secret of God was upon my tabernacle;
  When the Almighty was yet with me, when my children were about me;
  When I washed my steps with butter, and the rock poured me out rivers of oil;
  When I went out to the gate through the city, when I prepared my seat in the street! Behold, I go forward, but he is not there; and backward, but I cannot perceive him:
  On the left hand, where he doth work, but I cannot behold him: he hideth himself on the right hand, that I cannot see him:
  But he knoweth the way that I take: when he hath tried me, I shall come forth as gold. Surely thou hast spoken in mine hearing, and I have heard the 

In [101]:
summarization_pipeline_consolidation(job)

' Job arose, and rent his mantle, and shaved his head, and fell down upon the ground, and worshipped,  And said, Naked came I out of my mother\'s womb, and naked shall I return thither . Satan smote Job with sore boils from the sole of his foot unto his crown . He took him a potsherd to scrape himself withal; and he sat down among the ashes . The Chaldeans made out three bands and fell upon the camels, and have carried them away, yea, and slain the servants . But Job answered: \'Oh that my grief were throughly weighed, and my calamity laid in the balances together! For now it would be heavier than the sand of the sea: therefore my words are swallowed up\' Bildad the Shuhite said, \'How long wilt  Bildad the Shuhite said, "My breath is corrupt, my days are extinct, the graves are ready for me" Bildad replied, "Wherefore are we counted as beasts, and reputed vile in your sight?" Bildad then said, \'How long will it be ere ye make an end of words? mark, and afterwards we will speak?\' The

## Psalms

In [53]:
psalms = get_book_text('Psalms', bible)

In [153]:
printing(psalms)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Praise the LORD from the earth, ye dragons, and all deeps:
  Fire, and hail; snow, and vapour; stormy wind fulfilling his word:
  Mountains, and all hills; fruitful trees, and all cedars:
  Beasts, and all cattle; creeping things, and flying fowl:
  Kings of the earth, and all people; princes, and all judges of the earth:
  Both young men, and maidens; old men, and children:
  Let them praise the name of the LORD: for his name alone is excellent; his glory is above the earth and heaven. It is a good thing to give thanks unto the LORD, and to sing praises unto thy name, O most High:
  To shew forth thy lovingkindness in the morning, and thy faithfulness every night,
  Upon an instrument of ten strings, and upon the psaltery; upon the harp with a solemn sound. For thou hast delivered my soul from death: wilt not thou deliver my feet from falling, that I may walk before God in 

In [104]:
summarization_pipeline_consolidation(psalms)

' A Psalm of David was written by the chief Musician on Neginoth upon Sheminith . He sang it to the people in Shiggaion of David, concerning the words of Cush the Benjamite . My defence is of God, which saveth the upright in heart. Offer the sacrifices of righteousness, and put your trust in the LORD . There be many that say, Who will shew us any good? But the LORD, O LORD, art a shield for me; my glory, and the lifter up of mine head. I cried unto the LORD with my voice, and he heard me out of his holy hill . Have mercy upon me, O Lord, consider my trouble which I suffer of them that hate me, thou that liftest me up from the gates of death:  A Psalm of David was written by the chief Musician upon Shoshannim, a Psalm . The singers went before, the players on instruments followed after; among them were the damsels playing with timbrels . In it, he says: "Let the people praise thee, O God," then the earth will yield her increase; and God, even our own God, shall bless us" But I am a stra

## Proverbs

In [55]:
proverbs = get_book_text('Proverbs', bible)

In [56]:
printing(proverbs)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
When wisdom entereth into thine heart, and knowledge is pleasant unto thy soul;
  Discretion shall preserve thee, understanding shall keep thee:
  To deliver thee from the way of the evil man, from the man that speaketh froward things;
  Who leave the paths of uprightness, to walk in the ways of darkness;
  Who rejoice to do evil, and delight in the frowardness of the wicked;
  Whose ways are crooked, and they froward in their paths:
  To deliver thee from the strange woman, even from the stranger which flattereth with her words;
  Which forsaketh the guide of her youth, and forgetteth the covenant of her God. She hath sent forth her maidens: she crieth upon the highest places of the city,
  Whoso is simple, let him turn in hither: as for him that wanteth understanding, she saith to him,
  Come, eat of my bread, and drink of the wine which I have mingled. When he prepared th

In [107]:
summarization_pipeline_consolidation(proverbs)

' Solomon the son of David, king of Israel, gives subtilty to the simple . The fear of the LORD is the beginning of knowledge: but fools despise wisdom and instruction . A naughty person, a wicked man, walketh with a froward mouth . Frowardness is in his heart, he deviseth mischief continually; he soweth discord . So shall thy poverty come as one that travelleth, and thy want as an armed man . Lust not after her beauty in thine heart; neither let her take thee with her eyelids . Men do not despise a thief, if he steal to satisfy his soul when he is hungry . If he be found, he shall restore sevenfold; he shall give all the substance of his house . He that hid hatred with lying lips, and he uttereth a slander, is a fool  Deceit is in the heart of them that imagine evil: but to counsellors of peace is joy . Lying lips are abomination to the LORD: but they that deal truly are his delight . As righteousness tendeth to life, so he that pursueth evil pursueth it to his own death . Behold, the

## Ecclesiastes

In [57]:
ecclesiastes = get_book_text('Ecclesiastes', bible)

In [58]:
printing(ecclesiastes)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Remember now thy Creator in the days of thy youth, while the evil days come not, nor the years draw nigh, when thou shalt say, I have no pleasure in them;
  While the sun, or the light, or the moon, or the stars, be not darkened, nor the clouds return after the rain:
  In the day when the keepers of the house shall tremble, and the strong men shall bow themselves, and the grinders cease because they are few, and those that look out of the windows be darkened,
  And the doors shall be shut in the streets, when the sound of the grinding is low, and he shall rise up at the voice of the bird, and all the daughters of musick shall be brought low;
  Also when they shall be afraid of that which is high, and fears shall be in the way, and the almond tree shall flourish, and the grasshopper shall be a burden, and desire shall fail: because man goeth to his long home, and the mourners

In [110]:
summarization_pipeline_consolidation(ecclesiastes)

" The words of the Preacher, the son of David, king in Jerusalem, are said to be the words of a man who was king over Israel in Jerusalem . All things are full of labour; man cannot utter it: the eye is not satisfied with seeing, nor the ear filled with hearing . One generation passeth away, and another generation cometh: but the earth abideth for ever . There is no remembrance of former things; neither shall there be any remembrance of things that are to come with those that will come after . I communed with mine own heart, saying, Lo, I am come to great estate, and have gotten more wisdom than all they that have been before me in Jerusalem: yea, my heart had great experience of wisdom and knowledge . So I was great, and increased more than all   I said in mine heart, God shall judge the righteous and the wicked: for there is a time there for every purpose and for every work . Then I returned, and I saw vanity under the sun: and behold the tears of such as were oppressed, and they had

## Song of Solomon

In [59]:
song_of_solomon = get_book_text('Song of Solomon', bible)

In [60]:
printing(song_of_solomon)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
I said, I will go up to the palm tree, I will take hold of the boughs thereof: now also thy breasts shall be as clusters of the vine, and the smell of thy nose like apples;
  And the roof of thy mouth like the best wine for my beloved, that goeth down sweetly, causing the lips of those that are asleep to speak. I sleep, but my heart waketh: it is the voice of my beloved that knocketh, saying, Open to me, my sister, my love, my dove, my undefiled: for my head is filled with dew, and my locks with the drops of the night. I will rise now, and go about the city in the streets, and in the broad ways I will seek him whom my soul loveth: I sought him, but I found him not. I am black, but comely, O ye daughters of Jerusalem, as the tents of Kedar, as the curtains of Solomon. Thy plants are an orchard of pomegranates, with pleasant fruits; camphire, with spikenard,
  Spikenard and sa

In [113]:
summarization_pipeline_consolidation(song_of_solomon)

' A bundle of myrrh is my wellbeloved unto me; he shall lie all night betwixt my breasts . Stay me with flagons, comfort me with apples: for I am sick of love . My beloved is white and ruddy, the chiefest among ten thousand . His left hand is under my head and his right hand doth  The voice of my beloved! behold, he cometh leaping upon the mountains, skipping upon the hills . I charge you, O daughters of Jerusalem, by the roes, and by the hinds of the field, that ye stir not up, nor awake my love, till he please . For, lo, the winter is past, the rain is over and gone; the flowers appear on the earth; the time of the singing of birds is come, and the voice of the turtle is heard in  My beloved is gone down into his garden, to the beds of spices, to feed in the gardens, and to gather lilies . His eyes are as the eyes of doves by the rivers of waters, washed with milk, and fitly set . His hands are as gold rings set with the beryl: his belly is as bright ivory overlaid with sapphires . H

## Isaiah

In [61]:
isaiah = get_book_text('Isaiah', bible)

In [62]:
printing(isaiah)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Thus saith the LORD, thy redeemer, and he that formed thee from the womb, I am the LORD that maketh all things; that stretcheth forth the heavens alone; that spreadeth abroad the earth by myself;
  That frustrateth the tokens of the liars, and maketh diviners mad; that turneth wise men backward, and maketh their knowledge foolish;
  That confirmeth the word of his servant, and performeth the counsel of his messengers; that saith to Jerusalem, Thou shalt be inhabited; and to the cities of Judah, Ye shall be built, and I will raise up the decayed places thereof:
  That saith to the deep, Be dry, and I will dry up thy rivers:
  That saith of Cyrus, He is my shepherd, and shall perform all my pleasure: even saying to Jerusalem, Thou shalt be built; and to the temple, Thy foundation shall be laid. The Spirit of the Lord GOD is upon me; because the LORD hath anointed me to preach 

In [116]:
summarization_pipeline_consolidation(isaiah)

" The day of the LORD cometh, cruel both with wrath and fierce anger, to lay the land desolate . Babylon, the glory of kingdoms, the beauty of the Chaldees' excellency, shall be as when God overthrew Sodom and Gomorrah . And I will punish the world for their evil, and the wicked for their iniquity; and I will cause the arrogancy of the proud to cease, and will lay low the haughtiness of the terrible . Moab will be a covert to the outcasts from the face of the spoiler: for the extortioner is at an end, the spoiler ceaseth . This is the word that the LORD has spoken concerning Moab since that time . After the end of seventy years shall Tyre sing as an harlot, according to the days of one king  In the Bible, she says:  I have refined thee, but not with silver; I have chosen thee in the furnace of affliction . The loss of children and widowhood will come upon her in a moment in one day, the loss of thine children, or widowhood . She warns that evil will fall upon her 'and desolation shall 

## Jeremiah

In [63]:
jeremiah = get_book_text('Jeremiah', bible)

In [154]:
printing(jeremiah)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
behold, thou hast made the heaven and the earth by thy great power and stretched out arm, and there is nothing too hard for thee:
  Thou shewest lovingkindness unto thousands, and recompensest the iniquity of the fathers into the bosom of their children after them: the Great, the Mighty God, the LORD of hosts, is his name,
  Great in counsel, and mighty in work: for thine eyes are open upon all the ways of the sons of men: to give every one according to his ways, and according to the fruit of his doings:
  Which hast set signs and wonders in the land of Egypt, even unto this day, and in Israel, and among other men; and hast made thee a name, as at this day;
  And hast brought forth thy people Israel out of the land of Egypt with signs, and with wonders, and with a strong hand, and with a stretched out arm, and with great terror;
  And hast given them this land, which thou di

In [119]:
summarization_pipeline_consolidation(jeremiah)

' Jeremiah the son of Hilkiah, of the priests that were in Anathoth in the land of Benjamin . Return, ye backsliding children, and I will heal your backslidings, he says . For this gird you with sackcloth, lament and howl: for the fierce anger is not turned back from us . Circumcise yourselves to the LORD, and take away the foreskins of your heart, O Jerusalem, wash thine heart from wickedness, that thou mayest be saved . The carcases of this people shall be meat for the fowls of the heaven, and for the beasts of the earth; and none shall fray them away . Then will I cause to cease from the cities of Judah, and from the streets of Jerusalem, the voice of mirth, and the voices of gladness, will cease .  The days come, saith the LORD, that I will bring again the captivity of my people Israel and Judah . I will cause them to return to the land that I gave to their fathers, and they shall possess it . But they shall serve the LORD their God, and David their king, whom I will raise up unto 

## Lamentations

In [65]:
lamentations = get_book_text('Lamentations', bible)

In [66]:
printing(lamentations)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
The young and the old lie on the ground in the streets: my virgins and my young men are fallen by the sword; thou hast slain them in the day of thine anger; thou hast killed, and not pitied. He hath bent his bow like an enemy: he stood with his right hand as an adversary, and slew all that were pleasant to the eye in the tabernacle of the daughter of Zion: he poured out his fury like fire. They cried unto them, Depart ye; it is unclean; depart, depart, touch not: when they fled away and wandered, they said among the heathen, They shall no more sojourn there. It is good that a man should both hope and quietly wait for the salvation of the LORD. I called for my lovers, but they deceived me: my priests and mine elders gave up the ghost in the city, while they sought their meat to relieve their souls. Jerusalem hath grievously sinned; therefore she is removed: all that honoured 

In [122]:
summarization_pipeline_consolidation(lamentations)

' Judah is gone into captivity because of affliction, and because of great servitude . She was great among the nations, and princess among the provinces, how is she become tributary! She weepeth sore in the night, and her tears are on her cheeks: among all her lovers she hath none to comfort her . O LORD, behold my affliction: for the enemy hath magnified himself . The yoke of my transgressions is bound by his hand: they are wreathed, and come up upon my neck . For these things I weep; mine eye, mine eye runneth down with water, because the comforter that should relieve my soul is far from me . The Lord trodden under foot all my mighty men in the midst of me: he hath called an assembly against me to crush my young men  The daughter of my people is become cruel, like the ostriches in the wilderness . The tongue of the sucking child cleaveth to the roof of his mouth for thirst . They that did feed delicately are desolate in the streets: they that were brought up in scarlet embrace dunghi

## Ezekiel

In [67]:
ezekiel = get_book_text('Ezekiel', bible)

In [68]:
printing(ezekiel)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Also, thou son of man, prophesy unto the mountains of Israel, and say, Ye mountains of Israel, hear the word of the LORD:
  Thus saith the Lord GOD; Because the enemy hath said against you, Aha, even the ancient high places are ours in possession:
  Therefore prophesy and say, Thus saith the Lord GOD; Because they have made you desolate, and swallowed you up on every side, that ye might be a possession unto the residue of the heathen, and ye are taken up in the lips of talkers, and are an infamy of the people:
  Therefore, ye mountains of Israel, hear the word of the Lord GOD; Thus saith the Lord GOD to the mountains, and to the hills, to the rivers, and to the valleys, to the desolate wastes, and to the cities that are forsaken, which became a prey and derision to the residue of the heathen that are round about;
  Therefore thus saith the Lord GOD; Surely in the fire of my 

In [125]:
summarization_pipeline_consolidation(ezekiel)

" Ezekiel the priest, the son of Buzi, in the land of the Chaldeans by the river Chebar . A whirlwind came out of the north, a great cloud, and a fire infolding itself . Out of the midst thereof came the likeness of four living creatures . Their appearance was like burning coals of fire, and like the appearance of lamps . As for their likeness, they four had the face of a man, and the faces of a lion, on the right side; and they also had an eagle . When they went, they went upon their four sides: and they turned not when they went . A third part of thee shall die with the pestilence, and with famine shall they be consumed in the midst of thee . The whole remnant of thee will I scatter into all the winds, and I will draw out a sword after them .  In thee have they set light by father and mother: in the midst of thee have dealt by oppression with the stranger . One has committed abomination with his neighbour's wife; another has lewdly defiled his daughter in law . And I will scatter the

## Daniel

In [69]:
daniel = get_book_text('Daniel', bible)

In [70]:
printing(daniel)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Now the queen, by reason of the words of the king and his lords, came into the banquet house: and the queen spake and said, O king, live for ever: let not thy thoughts trouble thee, nor let thy countenance be changed:
  There is a man in thy kingdom, in whom is the spirit of the holy gods; and in the days of thy father light and understanding and wisdom, like the wisdom of the gods, was found in him; whom the king Nebuchadnezzar thy father, the king, I say, thy father, made master of the magicians, astrologers, Chaldeans, and soothsayers;
  Forasmuch as an excellent spirit, and knowledge, and understanding, interpreting of dreams, and shewing of hard sentences, and dissolving of doubts, were found in the same Daniel, whom the king named Belteshazzar: now let Daniel be called, and he will shew the interpretation. Then an herald cried aloud, To you it is commanded, O people, n

In [128]:
summarization_pipeline_consolidation(daniel)

" Daniel, Hananiah, Mishael, and Azariah had understanding in all visions and dreams . They were ten times better than magicians and astrologers that were in all his realm . Daniel purposed in his heart that he would not defile himself with the portion of the meat, nor with the wine which he drank; therefore he requested of the Prince of the Eunuchs that he might notdefile himself . The king made Daniel a great man, and gave him many great gifts, and made him ruler over the whole province of Babylon . Daniel is the spirit of the gods, and light and understanding and excellent wisdom is found in him . Daniel was preferred above the presidents and princes, because an excellent spirit was in him. But when his heart was lifted up, and his mind hardened in pride, he was deposed  Daniel prospered in the reign of Darius, and Cyrus the Persian . In the first year of Belshazzar king of Babylon Daniel had vision of four great beasts . One was like a lion, and had eagle's wings: I beheld till the

## Hosea

In [71]:
hosea = get_book_text('Hosea', bible)

In [72]:
printing(hosea)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Plead with your mother, plead: for she is not my wife, neither am I her husband: let her therefore put away her whoredoms out of her sight, and her adulteries from between her breasts;
  Lest I strip her naked, and set her as in the day that she was born, and make her as a wilderness, and set her like a dry land, and slay her with thirst. For the children of Israel shall abide many days without a king, and without a prince, and without a sacrifice, and without an image, and without an ephod, and without teraphim:
  Afterward shall the children of Israel return, and seek the LORD their God, and David their king; and shall fear the LORD and his goodness in the latter days. Ye have plowed wickedness, ye have reaped iniquity; ye have eaten the fruit of lies: because thou didst trust in thy way, in the multitude of thy mighty men. For she did not know that I gave her corn, and wi

In [131]:
summarization_pipeline_consolidation(hosea)

" Hosea, the son of Beeri, in the days of Uzziah, Jotham, Ahaz, and Hezekiah, kings of Judah,  kings of Israel . So he went and took Gomer the daughter of Diblaim; which conceived, and bare him a son . And she conceived again, and Bare a daughter. And God said, Call her name Loruhamah: for I will no more have mercy upon the house of Israel; but I will utterly take them away . Yet the number of the children of Israel shall be as the sand of the sea, which cannot be measured nor numbered . It shall come to pass at that day, that I will break the bow of Israel in the valley of Jezreel. But I will not save them by bow, nor by sword   Israel slideth back as a backsliding heifer: now the LORD will feed them as a lamb in a large place . The princes of Judah were like men that remove the bound: therefore I will pour out my wrath upon them like water . They have dealt treacherously against the LORD: for they have begotten strange children: now shall a month devour them with their portions . Blo

## Joel

In [73]:
joel = get_book_text('Joel', bible)

In [74]:
printing(joel)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Yea, the LORD will answer and say unto his people, Behold, I will send you corn, and wine, and oil, and ye shall be satisfied therewith: and I will no more make you a reproach among the heathen:
  But I will remove far off from you the northern army, and will drive him into a land barren and desolate, with his face toward the east sea, and his hinder part toward the utmost sea, and his stink shall come up, and his ill savour shall come up, because he hath done great things. Be not afraid, ye beasts of the field: for the pastures of the wilderness do spring, for the tree beareth her fruit, the fig tree and the vine do yield their strength. Blow the trumpet in Zion, sanctify a fast, call a solemn assembly:
  Gather the people, sanctify the congregation, assemble the elders, gather the children, and those that suck the breasts: let the bridegroom go forth of his chamber, and th

In [134]:
summarization_pipeline_consolidation(joel)

' The word of the LORD that came to Joel the son of Pethuel. Hear this, ye old men, and give ear, all ye inhabitants of the land. Tell ye your children of it, and let your children tell their children . For a nation is come up upon my land, strong, and without number, whose teeth are the teeth of a lion . He hath laid my vine waste, and barked my fig tree: he hath made it clean bare, and cast it away . Lament like a virgin girded with sackcloth for the husband of her youth. The field is wasted, the land mourneth; for the corn is wasted: the new wine is dried up, the oil languisheth . Be ye ashamed, O ye husbandmen; howl, O Ye vinedressers, for the  A great people and a strong; there hath not been ever the like, neither shall be any more after it, even to the years of many generations . The appearance of them is as the appearance of horses; and as horsemen, so shall they run . They shall run like mighty men; they shall climb the wall like men of war; and they shall march every one on hi

## Amos

In [75]:
amos = get_book_text('Amos', bible)

In [76]:
printing(amos)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Hear this word that the LORD hath spoken against you, O children of Israel, against the whole family which I brought up from the land of Egypt, saying,
  You only have I known of all the families of the earth: therefore I will punish you for all your iniquities. Thus saith the LORD; For three transgressions of Gaza, and for four, I will not turn away the punishment thereof; because they carried away captive the whole captivity, to deliver them up to Edom:
  But I will send a fire on the wall of Gaza, which shall devour the palaces thereof:
  And I will cut off the inhabitant from Ashdod, and him that holdeth the sceptre from Ashkelon, and I will turn mine hand against Ekron: and the remnant of the Philistines shall perish, saith the Lord GOD. for ye have turned judgment into gall, and the fruit of righteousness into hemlock:
  Ye which rejoice in a thing of nought, which say

In [137]:
summarization_pipeline_consolidation(amos)

' Amos, who was among the herdmen of Tekoa, said he saw concerning Israel in the days of Uzziah king of Judah and Jeroboam the son of Joash king of Israel, two years before the earthquake . And he said, The LORD will roar from Zion, and utter his voice from Jerusalem; and the habitations of the shepherds shall mourn, and the top of Carmel shall wither . But I will send a fire into the house of Hazael, which shall devour the palaces of Benhadad. I will break also the bar of Damascus, and cut off the inhabitant from the plain of Aven . And the people of Syria shall go into captivity unto Kir . For three transgressions of the children of Ammon,  I will not turn away the punishment thereof; because they have ripped up the women with child of Gilead .  The virgin of Israel is fallen; she shall no more rise: she is forsaken upon her land; there is none to raise her up . And ye shall go out at the breaches, every cow at that which is before her; and ye shall cast them into the palace, saith t

## Obadiah

In [77]:
obadiah = get_book_text('Obadiah', bible)

In [78]:
printing(obadiah)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
For the day of the LORD is near upon all the heathen: as thou hast done, it shall be done unto thee: thy reward shall return upon thine own head. how are his hidden things sought up! The vision of Obadiah. would they not have stolen till they had enough? if the grapegatherers came to thee, would they not leave some grapes? And thy mighty men, O Teman, shall be dismayed, to the end that every one of the mount of Esau may be cut off by slaughter. And the house of Jacob shall be a fire, and the house of Joseph a flame, and the house of Esau for stubble, and they shall kindle in them, and devour them; and there shall not be any remaining of the house of Esau; for the LORD hath spoken it. Thou shouldest not have entered into the gate of my people in the day of their calamity; yea, thou shouldest not have looked on their affliction in the day of their calamity, nor have laid hands

In [140]:
summarization_pipeline_consolidation(obadiah)

"The vision of Obadiah. Thus saith the Lord GOD concerning Edom; We have heard a rumour from the LORD, and an ambassador is sent among the heathen, Arise ye, and let us rise up against her in battle.\n  Behold, I have made thee small among the heathen: thou art greatly despised.\n  The pride of thine heart hath deceived thee, thou that dwellest in the clefts of the rock, whose habitation is high; that saith in his heart, Who shall bring me down to the ground?\n  Though thou exalt thyself as the eagle, and though thou set thy nest among the stars, thence will I bring thee down, saith the LORD.\n  If thieves came to thee, if robbers by night, (how art thou cut off!) would they not have stolen till they had enough? if the grapegatherers came to thee, would they not leave some grapes?\n  How are the things of Esau searched out! how are his hidden things sought up!\n  All the men of thy confederacy have brought thee even to the border: the men that were at peace with thee have deceived thee

## Jonah

In [79]:
jonah = get_book_text('Jonah', bible)

In [155]:
printing(jonah)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Therefore I fled before unto Tarshish: for I knew that thou art a gracious God, and merciful, slow to anger, and of great kindness, and repentest thee of the evil. And Jonah began to enter into the city a day's journey, and he cried, and said, Yet forty days, and Nineveh shall be overthrown. Nevertheless the men rowed hard to bring it to the land; but they could not: for the sea wrought, and was tempestuous against them. And it came to pass, when the sun did arise, that God prepared a vehement east wind; and the sun beat upon the head of Jonah, that he fainted, and wished in himself to die, and said, It is better for me to die than to live. And he caused it to be proclaimed and published through Nineveh by the decree of the king and his nobles, saying, Let neither man nor beast, herd nor flock, taste any thing: let them not feed, nor drink water:
  But let man and beast be c

In [143]:
summarization_pipeline_consolidation(jonah)

' The word of the LORD came unto Jonah the son of Amittai, saying,  Arise, go to Nineveh, that great city, and cry against it; for their wickedness is come up before me . But Jonah rose up to flee unto Tarshish from the presence of the Lord, and went down to Joppa . He found a ship with them, and he went down into it, to go with them unto Tashish . But the LORD sent out a great wind into the sea, and there was a mighty tempest in the sea so that the ship was like to be broken . Then the mariners were afraid, and cried every man unto his god, and cast forth the wares that were in the ship into the Sea, to lighten it of them. But Jonah was gone down into the ship  Jonah was in the belly of a great fish for three days and three nights . He said, "I cried by reason of mine affliction unto the LORD, and he heard me" The people of Nineveh believed God and proclaimed a fast, and put on sackcloth, from the greatest of them even to the least of them . And the word of the LORD came unto Jonah th

## Micah

In [81]:
micah = get_book_text('Micah', bible)

In [82]:
printing(micah)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And many nations shall come, and say, Come, and let us go up to the mountain of the LORD, and to the house of the God of Jacob; and he will teach us of his ways, and we will walk in his paths: for the law shall go forth of Zion, and the word of the LORD from Jerusalem. Woe to them that devise iniquity, and work evil upon their beds! If a man walking in the spirit and falsehood do lie, saying, I will prophesy unto thee of wine and of strong drink; he shall even be the prophet of this people. Therefore I will make Samaria as an heap of the field, and as plantings of a vineyard: and I will pour down the stones thereof into the valley, and I will discover the foundations thereof. And it shall come to pass in that day, saith the LORD, that I will cut off thy horses out of the midst of thee, and I will destroy thy chariots:
  And I will cut off the cities of thy land, and throw do

In [146]:
summarization_pipeline_consolidation(micah)

' The word of the LORD that came to Micah the Morasthite in the days of Jotham, Ahaz, and Hezekiah, kings of Judah, which he saw concerning Samaria and Jerusalem . What is the transgression of Jacob? is it not Samaria? and what are the high places of Judah? are they not Jerusalem? Therefore I will make Samaria as an heap of the field, and as plantings of a vineyard . And I will pour down the stones thereof into the valley, and I will discover the foundations thereof . And all the graven images thereof shall be beaten to pieces, and all the hires thereof shall burn with the fire . Declare ye it not at Gath, weep ye not at all: in the house of Aphrah roll thyself in the dust .  The remnant of Jacob shall be in the midst of many people as a dew from the LORD, as the showers upon the grass, that tarrieth not for man, nor waiteth for the sons of men . And I will make her that halted a remnant, and her that was cast far off a strong nation: and the LORD shall reign over them in mount Zion fr

## Nahum

In [83]:
nahum = get_book_text('Nahum', bible)

In [84]:
printing(nahum)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Behold, I am against thee, saith the LORD of hosts, and I will burn her chariots in the smoke, and the sword shall devour thy young lions: and I will cut off thy prey from the earth, and the voice of thy messengers shall no more be heard. Thou also shalt be drunken: thou shalt be hid, thou also shalt seek strength because of the enemy. He rebuketh the sea, and maketh it dry, and drieth up all the rivers: Bashan languisheth, and Carmel, and the flower of Lebanon languisheth. Take ye the spoil of silver, take the spoil of gold: for there is none end of the store and glory out of all the pleasant furniture. Who can stand before his indignation? his fury is poured out like fire, and the rocks are thrown down by him. She is empty, and void, and waste: and the heart melteth, and the knees smite together, and much pain is in all loins, and the faces of them all gather blackness. Th

In [149]:
summarization_pipeline_consolidation(nahum)

' The LORD is slow to anger, and great in power, and will not acquit the wicked . He rebuketh the sea, and maketh it dry, and drieth up all the rivers: Bashan languisheth, and Carmel, and the flower of Lebanon languishesheth . But with an overrunning flood he will make an utter end of the place thereof, and darkness shall pursue his enemies . For while they be folden together as thorns, and while they are drunken as drunkards, they shall be devoured as stubble fully dry. There is one come out of thee, that imagineth evil against the LORD, a wicked counsellor. Though I have afflicted thee, I will afflict thee no more. For now will I break his yoke from off thee,  Nineveh is empty, void, and waste: and the heart melteth, and the knees smite together, and much pain is in all loins . Huzzab shall be led away captive, she shall be brought up, and her maids shall lead her as with the voice of doves, tabering upon their breasts . The chariots shall rage in the streets, they shall justle one a

## Habakkuk

In [85]:
habakkuk = get_book_text('Habakkuk', bible)

In [86]:
printing(habakkuk)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Thou art filled with shame for glory: drink thou also, and let thy foreskin be uncovered: the cup of the LORD'S right hand shall be turned unto thee, and shameful spewing shall be on thy glory. He stood, and measured the earth: he beheld, and drove asunder the nations; and the everlasting mountains were scattered, the perpetual hills did bow: his ways are everlasting. What profiteth the graven image that the maker thereof hath graven it; the molten image, and a teacher of lies, that the maker of his work trusteth therein, to make dumb idols? Before him went the pestilence, and burning coals went forth at his feet. God came from Teman, and the Holy One from mount Paran. Woe to him that coveteth an evil covetousness to his house, that he may set his nest on high, that he may be delivered from the power of evil! for spoiling and violence are before me: and there are that raise 

In [152]:
summarization_pipeline_consolidation(habakkuk)

" The burden which Habakkuk the prophet did see. The burden of the prophet . The burden he did see . Behold ye among the heathen, and regard, and wonder marvellously: for I will work a work in your days, which ye will not believe, though it be told you . For, lo, I raise up the Chaldeans, that bitter and hasty nation, which shall march through the breadth of the land, to possess the dwellingplaces that are not theirs. They are terrible and dreadful: their judgment and their dignity shall proceed of themselves. Their horses also are swifter than the leopards, and are more fierce than the evening wolves . Then shall his mind change, and he shall pass over, and offend, imputing this his power unto his god  I will stand upon my watch, and set me upon the tower, and will watch to see what he will say unto me . And the LORD answered me, and said, Write the vision, and make it plain upon tables . For the vision is yet for an appointed time, but at the end it shall speak, and not lie: though i

## Zephaniah

In [87]:
zephaniah = get_book_text('Zephaniah', bible)

In [88]:
printing(zephaniah)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Therefore wait ye upon me, saith the LORD, until the day that I rise up to the prey: for my determination is to gather the nations, that I may assemble the kingdoms, to pour upon them mine indignation, even all my fierce anger: for all the earth shall be devoured with the fire of my jealousy. The LORD hath taken away thy judgments, he hath cast out thine enemy: the king of Israel, even the LORD, is in the midst of thee: thou shalt not see evil any more. I have heard the reproach of Moab, and the revilings of the children of Ammon, whereby they have reproached my people, and magnified themselves against their border. And he will stretch out his hand against the north, and destroy Assyria; and will make Nineveh a desolation, and dry like a wilderness. Her prophets are light and treacherous persons: her priests have polluted the sanctuary, they have done violence to the law. Fr

In [155]:
summarization_pipeline_consolidation(zephaniah)

' I will utterly consume all things from off the land, saith the LORD . I will also stretch out mine hand upon Judah, and upon all the inhabitants of Jerusalem . And it shall come to pass at that time, that I will search Jerusalem with candles, and punish the men that are settled on their lees: that say in their heart, The LORD will not do good, neither will he do evil. Therefore their goods shall become a booty, and their houses will become a desolation: they shall also build houses, but not inhabit them; and they shall plant vineyards, \xa0not drink the wine thereof. That day is a day of wrath, a day ... A day of wasteness and desolation, a great crashing from the hills. Howl, ye inhabitants of Maktesh, for all the merchant people are cut down; all they that bear silver  O Canaan, the land of the Philistines, I will even destroy thee, that there shall be no inhabitant . Gaza shall be forsaken, and Ashkelon a desolation: they shall drive out Ashdod at the noon day, and Ekron shall be 

## Haggai

In [89]:
haggai = get_book_text('Haggai', bible)

In [90]:
printing(haggai)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Yet now be strong, O Zerubbabel, saith the LORD; and be strong, O Joshua, son of Josedech, the high priest; and be strong, all ye people of the land, saith the LORD, and work: for I am with you, saith the LORD of hosts:
  According to the word that I covenanted with you when ye came out of Egypt, so my spirit remaineth among you: fear ye not. And the priests answered and said, It shall be unclean. yea, as yet the vine, and the fig tree, and the pomegranate, and the olive tree, hath not brought forth: from this day will I bless you. And again the word of the LORD came unto Haggai in the four and twentieth day of the month, saying,
  Speak to Zerubbabel, governor of Judah, saying, I will shake the heavens and the earth;
  And I will overthrow the throne of kingdoms, and I will destroy the strength of the kingdoms of the heathen; and I will overthrow the chariots, and those tha

In [158]:
summarization_pipeline_consolidation(haggai)

" In the second year of Darius the king,  in the sixth month, in the first day of the month, came the word of the LORD by Haggai the prophet unto Zerubbabel the son of Shealtiel, governor of Judah, and to Joshua the . son of Josedech, the high priest, saying, This people say, The time is not come, the time that the LORD'S house should be built . Now therefore thus saith the LORD of hosts; Consider your ways. Go up to the mountain, and bring wood, and build the house; and I will take pleasure in it, and ... I will be glorified, saath the LORD . Ye looked for much, and, lo, it came to little; and when ye brought  In the seventh month, in the one and twentieth day of the month, came the word of the LORD by the prophet Haggai, saying, If one bear holy flesh in the skirt of his garment, and with his skirt do touch bread, or pottage, or wine, or oil, or any meat, shall it be holy? And the priests answered and said, No. If one that is unclean by a dead body touch any of these, will it be uncl

## Zechariah

In [91]:
zechariah = get_book_text('Zechariah', bible)

In [156]:
printing(zechariah)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And the word of the LORD came unto me, saying,
  Take of them of the captivity, even of Heldai, of Tobijah, and of Jedaiah, which are come from Babylon, and come thou the same day, and go into the house of Josiah the son of Zephaniah;
  Then take silver and gold, and make crowns, and set them upon the head of Joshua the son of Josedech, the high priest;
  And speak unto him, saying, Thus speaketh the LORD of hosts, saying, Behold the man whose name is The BRANCH; and he shall grow up out of his place, and he shall build the temple of the LORD:
  Even he shall build the temple of the LORD; and he shall bear the glory, and shall sit and rule upon his throne; and he shall be a priest upon his throne: and the counsel of peace shall be between them both. But my words and my statutes, which I commanded my servants the prophets, did they not take hold of your fathers? Then the ange

In [161]:
summarization_pipeline_consolidation(zechariah)

" In the eighth month, in the second year of Darius, came the word of the LORD unto Zechariah, the son of Iddo the prophet, saying, The LORD hath been sore displeased with your fathers . But my words and my statutes, which I commanded my servants the prophets, did they not take hold of your fathers? Then a man riding upon a red horse, and he stood among the myrtle trees that were in the bottom; and behind him were there red horses, speckled, and white . Then said I, O my lord, what are these? And the angel that communed with me said unto me. These are they  I am returned to Jerusalem with mercies: my house shall be built in it, saith the LORD of hosts, and a line shall be stretched forth upon Jerusalem .  The word of the LORD came unto Zechariah in the fourth day of the ninth month, even in Chisleu . And he said, This is the curse that goeth forth over the face of the whole earth: for every one that stealeth shall be cut off as on this side according to it . And I turned, and lifted up

## Malachi

In [93]:
malachi = get_book_text('Malachi', bible)

In [94]:
printing(malachi)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And they shall be mine, saith the LORD of hosts, in that day when I make up my jewels; and I will spare them, as a man spareth his own son that serveth him. Because the LORD hath been witness between thee and the wife of thy youth, against whom thou hast dealt treacherously: yet is she thy companion, and the wife of thy covenant. But ye have profaned it, in that ye say, The table of the LORD is polluted; and the fruit thereof, even his meat, is contemptible. The law of truth was in his mouth, and iniquity was not found in his lips: he walked with me in peace and equity, and did turn many away from iniquity. saith the LORD: yet I loved Jacob,
  And I hated Esau, and laid his mountains and his heritage waste for the dragons of the wilderness. And now we call the proud happy; yea, they that work wickedness are set up; yea, they that tempt God are even delivered. Who is there ev

In [164]:
summarization_pipeline_consolidation(malachi)

" The burden of the word of the LORD to Israel by Malachi. I have loved you, saith the LORD . Yet yet I loved Jacob, and laid his mountains and his heritage waste for the dragons of the wilderness . And if ye offer the blind for sacrifice, is it not evil? offer it now unto thy governor; will he be pleased with thee, or accept thy person?  Offer polluted bread upon mine altar; and ye say, Wherein have we despised thy name? But ye have profaned it, in that ye say: The table of the Lord is contemptible . For my name shall be great among the heathen, saeth the LORD of hosts. But Ye said also, Behold, what a weariness is it! and ye have snuffed at it, saath the LORD’s  The LORD will cut off the man that doeth this, the master and scholar, out of the tabernacles of Jacob, and him that offereth an offering unto the LORD of hosts . But cursed be the deceiver, which hath in his flock a male, and voweth, and sacrificeth unto the Lord a corrupt thing . For the priest's lips should keep knowledge,

# New Testament

## Full New Testament

In [96]:
printing(new_testament)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And you, that were sometime alienated and enemies in your mind by wicked works, yet now hath he reconciled
  In the body of his flesh through death, to present you holy and unblameable and unreproveable in his sight:
  If ye continue in the faith grounded and settled, and be not moved away from the hope of the gospel, which ye have heard, and which was preached to every creature which is under heaven; whereof I Paul am made a minister;
  Who now rejoice in my sufferings for you, and fill up that which is behind of the afflictions of Christ in my flesh for his body's sake, which is the church:
  Whereof I am made a minister, according to the dispensation of God which is given to me for you, to fulfil the word of God;
  Even the mystery which hath been hid from ages and from generations, but now is made manifest to his saints:
  To whom God would make known what is the riches 

In [167]:
summarization_pipeline_consolidation(new_testament)

' The birth of Jesus Christ was on this wise: When as his mother Mary was espoused to Joseph, before they came together, she was found with child of Holy Ghost . After they were brought to Babylon, Jechonias begat Salathiel; and after they had been brought to\xa0Babylon\xa0are 14 generations . So all the generations from Abraham to David are fourteen generations . In fact, the Bible had no conception of Jesus as an adult when he became a man and godfather . It says Jesus was born into a family of men, not just children, but adults; they were also parents . They are responsible for the Bible’s origin; it says: “The Bible” – A generation of people have come to their faith . He explains how the Bible\'s creation of the Bible changed the way  The first day of the week cometh Mary Magdelene seeth the stone taken away from the sepulchre . He breathed on them, Receive ye the Holy Ghost: Whose soever sins ye remit, they are remitted unto them . They were amazed and marvelled, saying one to ano

## Matthew

In [97]:
matthew = get_book_text('Matthew', bible)

In [98]:
printing(matthew)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Now when Jesus had heard that John was cast into prison, he departed into Galilee;
  And leaving Nazareth, he came and dwelt in Capernaum, which is upon the sea coast, in the borders of Zabulon and Nephthalim:
  That it might be fulfilled which was spoken by Esaias the prophet, saying,
  The land of Zabulon, and the land of Nephthalim, by the way of the sea, beyond Jordan, Galilee of the Gentiles;
  The people which sat in darkness saw great light; and to them which sat in the region and shadow of death light is sprung up. He answered and said unto them, He that soweth the good seed is the Son of man;
  The field is the world; the good seed are the children of the kingdom; but the tares are the children of the wicked one;
  The enemy that sowed them is the devil; the harvest is the end of the world; and the reapers are the angels. But and if that evil servant shall say in hi

In [170]:
summarization_pipeline_consolidation(matthew)

' The birth of Jesus Christ was on this wise: When as his mother Mary was espoused to Joseph, before they came together, she was found with child of Holy Ghost . And after they were brought to Babylon, Jechonias begat Salathiel; and  after they had been brought to\xa0Babylon\xa0are 14 generations . So all the generations from Abraham to David are fourteen generations . But when Herod was dead, behold, an angel of the lord appeareth in a dream to Joseph in Egypt, and said, Arise, and go into the land of Israel: for they are dead which sought the little child\'s life . Then Joseph arose, and took the small child and its mother, and came into the Land of Israel . Jesus, when he was baptized, went up straightway out of the water  Jesus said, Can the children of the bridechamber mourn as long as the bridegroom is with them? The disciple is not above his master, nor the servant above his lord . He said, \'I send you forth as sheep in the midst of wolves: be ye therefore wise as serpents, and

## Mark 

In [99]:
mark = get_book_text('Mark', bible)

In [100]:
printing(mark)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And Jesus answered and said, Verily I say unto you, There is no man that hath left house, or brethren, or sisters, or father, or mother, or wife, or children, or lands, for my sake, and the gospel's,
  But he shall receive an hundredfold now in this time, houses, and brethren, and sisters, and mothers, and children, and lands, with persecutions; and in the world to come eternal life. And he ordained twelve, that they should be with him, and that he might send them forth to preach,
  And to have power to heal sicknesses, and to cast out devils:
  And Simon he surnamed Peter;
  And James the son of Zebedee, and John the brother of James; and he surnamed them Boanerges, which is, The sons of thunder:
  And Andrew, and Philip, and Bartholomew, and Matthew, and Thomas, and James the son of Alphaeus, and Thaddaeus, and Simon the Canaanite,
  And Judas Iscariot, which also betrayed

In [173]:
summarization_pipeline_consolidation(mark)

" The beginning of the gospel of Jesus Christ, the Son of God;  As it is written in the prophets, Behold, I send my messenger before thy face, which shall prepare thy way before thee . And there went out unto him all the land of Judaea, and they of Jerusalem, and were all baptized of him in the river of Jordan, confessing their sins . And John was clothed with camel's hair, and with a girdle of a skin about his loins; and he did eat locusts and wild honey;  And preached, saying, There cometh one mightier than I after me, the latchet of whose shoes I am not worthy to stoop down and unloose . He rebuked the wind, and said unto the sea, Peace, be still .  When the daughter of the said Herodias came in, and danced, and pleased Herod and them that sat with him, the king said, Ask of me whatsoever thou wilt, and I will give it thee . And she went forth, and said, What shall I ask? And she said, The head of John the Baptist. And they went out, and preached that men should repent . And cast ou

## Luke

In [101]:
luke = get_book_text('Luke', bible)

In [102]:
printing(luke)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And his father Zacharias was filled with the Holy Ghost, and prophesied, saying,
  Blessed be the Lord God of Israel; for he hath visited and redeemed his people,
  And hath raised up an horn of salvation for us in the house of his servant David;
  As he spake by the mouth of his holy prophets, which have been since the world began:
  That we should be saved from our enemies, and from the hand of all that hate us;
  To perform the mercy promised to our fathers, and to remember his holy covenant;
  The oath which he sware to our father Abraham,
  That he would grant unto us, that we being delivered out of the hand of our enemies might serve him without fear,
  In holiness and righteousness before him, all the days of our life. And it came to pass afterward, that he went throughout every city and village, preaching and shewing the glad tidings of the kingdom of God: and the tw

In [176]:
summarization_pipeline_consolidation(luke)

" The Bible says Jesus was the son and father of a group of men and women . He explains how the Bible's creation of the Bible changed the way God dealt with the birth of Jesus Christ . In fact, the Bible had no conception of Jesus as an adult when he became a man and godfather . It says that Jesus was born into a family of men, not just children, but adults; they were also parents . They are responsible for the Bible’s origin; it says: “The Bible” – A generation of people have come to their faith . We say that the Bible contains many passages from the Bible . Some of them are believed to have fallen away from its teachings . And Jesus is the son who has been with the world since he came to his first appearance in the Bible  Many widows were in Israel in the days of Elias' When the heaven was shut up three years and six months, when famine was  He laid his hands on every one of the sick with divers diseases, and healed them . And devils also came out of many, crying out, and saying, 'Th

## John

In [103]:
john = get_book_text('John', bible)

In [104]:
printing(john)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
These words spake Jesus, and lifted up his eyes to heaven, and said, Father, the hour is come; glorify thy Son, that thy Son also may glorify thee:
  As thou hast given him power over all flesh, that he should give eternal life to as many as thou hast given him. Nicodemus saith unto them, (he that came to Jesus by night, being one of them,)
  Doth our law judge any man, before it hear him, and know what he doeth? But Mary stood without at the sepulchre weeping: and as she wept, she stooped down, and looked into the sepulchre,
  And seeth two angels in white sitting, the one at the head, and the other at the feet, where the body of Jesus had lain. Our fathers worshipped in this mountain; and ye say, that in Jerusalem is the place where men ought to worship. And many of the Samaritans of that city believed on him for the saying of the woman, which testified, He told me all tha

In [179]:
summarization_pipeline_consolidation(john)

' The next day John seeth Jesus coming unto him, and saith, Behold the Lamb of God, which taketh away the sin of the world . These things were done in Bethabara beyond Jordan, where John was baptizing . After these things came Jesus and his disciples into Judaea; and there he tarried with them, and baptized . Then cometh he to a city of Samaria, which is called Sychar, near to the parcel that Jacob gave to his son Joseph . Many Samaritans of that city believed on him for the saying of the woman . But Jesus did not commit himself to them, because he knew all men,  He said, My meat is to do the will of him . And many more believed because of his own word .  After these things Jesus walked in Galilee . Many of his disciples went back, and walked no more with him . The day following, when the people stood on the other side of the sea saw that there was none other boat there, save that one where into his disciples were entered, and that Jesus went not with his disciples into the boat, but t

## Acts

In [105]:
acts = get_book_text('Acts', bible)

In [106]:
printing(acts)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Then pleased it the apostles and elders, with the whole church, to send chosen men of their own company to Antioch with Paul and Barnabas; namely, Judas surnamed Barsabas, and Silas, chief men among the brethren:
  And they wrote letters by them after this manner; The apostles and elders and brethren send greeting unto the brethren which are of the Gentiles in Antioch and Syria and Cilicia:
  Forasmuch as we have heard, that certain which went out from us have troubled you with words, subverting your souls, saying, Ye must be circumcised, and keep the law: to whom we gave no such commandment:
  It seemed good unto us, being assembled with one accord, to send chosen men unto you with our beloved Barnabas and Paul,
  Men that have hazarded their lives for the name of our Lord Jesus Christ. For David speaketh concerning him, I foresaw the Lord always before my face, for he is o

In [182]:
summarization_pipeline_consolidation(acts)

' Matthias was chosen to be a witness with us of his resurrection . Matthias, Barsabas and Justus were appointed as the two apostles on the day of Pentecost . They were amazed and marvelled, saying one to another, Behold, are not all these which speak Galilaeans? And how hear we every man in our own tongue, wherein we were born? We do hear them speak in our tongues the wonderful works of God . A notable miracle of healing was done by them that is manifest to all them that dwell in Jerusalem, and we cannot deny it . But that it spread no further among the people, let us straitly threaten them, that they speak henceforth to no man in this name . This is the stone which was set at nought of you builders, which is  become the head of the corner  When he was full forty years old, it came into his heart to visit his brethren the children of Israel . And seeing one of them suffer wrong, he defended him, and avenged him that was oppressed, and smote the Egyptian . But he that did his neighbour

## Romans

In [107]:
romans = get_book_text('Romans', bible)

In [108]:
printing(romans)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Paul, a servant of Jesus Christ, called to be an apostle, separated unto the gospel of God,
  (Which he had promised afore by his prophets in the holy scriptures,)
  Concerning his Son Jesus Christ our Lord, which was made of the seed of David according to the flesh;
  And declared to be the Son of God with power, according to the spirit of holiness, by the resurrection from the dead:
  By whom we have received grace and apostleship, for obedience to the faith among all nations, for his name:
  Among whom are ye also the called of Jesus Christ:
  To all that be in Rome, beloved of God, called to be saints: Grace to you and peace from God our Father, and the Lord Jesus Christ. Wherefore, as by one man sin entered into the world, and death by sin; and so death passed upon all men, for that all have sinned:
  (For until the law sin was in the world: but sin is not imputed when 

In [185]:
summarization_pipeline_consolidation(romans)

' Paul: "To all that be in Rome, beloved of God,"  Grace to you and peace from God our Father, and the Lord Jesus Christ" So, as much as in me is, I am ready to preach the gospel to you that are at Rome also . Professing themselves to be wise, they became fools . And changed the glory of the uncorruptible God into an image made like to corruptible man, and to birds, and fourfooted beasts, and creeping things . The name of God is blasphemed among the Gentiles through you through you, as it is written . Both Jews and Gentiles are all under sin, says the Jewish leader . But if our unrighteousness commend the righteousness of God, what shall we say? Is God unrighteous who taketh vengeance?  There is no difference between the Jew and the Greek: for the same Lord over all is rich unto all that call upon him . How then shall they call on him in whom they have not believed? And how shall they believe in him of whom they haven\'t heard? But I say, Have they not heard? Yes verily, their sound we

## 1 Corinthians

In [109]:
corinthians1 = get_book_text('1 Corinthians', bible)

In [110]:
printing(corinthians1)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
I speak as to wise men; judge ye what I say. And God hath set some in the church, first apostles, secondarily prophets, thirdly teachers, after that miracles, then gifts of healings, helps, governments, diversities of tongues. For if any man see thee which hast knowledge sit at meat in the idol's temple, shall not the conscience of him which is weak be emboldened to eat those things which are offered to idols;
  And through thy knowledge shall the weak brother perish, for whom Christ died? For I verily, as absent in body, but present in spirit, have judged already, as though I were present, concerning him that hath so done this deed,
  In the name of our Lord Jesus Christ, when ye are gathered together, and my spirit, with the power of our Lord Jesus Christ,
  To deliver such an one unto Satan for the destruction of the flesh, that the spirit may be saved in the day of the L

In [188]:
summarization_pipeline_consolidation(corinthians1)

' Paul was called to be an apostle of Jesus Christ through the will of God, and Sosthenes our brother, was a member of the church of God which is at Corinth . Now I beseech you, brethren, by the name of our Lord Jesus Christ, that ye all speak the same thing, and that there be no divisions among you . If any man among you seem to be wise in this world, let him become a fool, that he may be wise . The body is not for fornication,  Every sin that a man doeth is without the body; but he that committeth fornicated sinneth against his own body . Dare any of you, having a matter against another, go to law before the unjust, and not before the saints? Do ye not know that the saints shall judge the world?  I protest by your rejoicing which I have in Christ Jesus our Lord, I die daily . I thank my God, I speak with tongues more than ye all:  Yet in the church I had rather speak five words with my understanding, that by my voice I might teach others also, than ten thousand words in an known tong

## 2 Corinthians

In [111]:
corinthians2 = get_book_text('2 Corinthians', bible)

In [157]:
printing(corinthians2)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Giving no offence in any thing, that the ministry be not blamed:
  But in all things approving ourselves as the ministers of God, in much patience, in afflictions, in necessities, in distresses,
  In stripes, in imprisonments, in tumults, in labours, in watchings, in fastings;
  By pureness, by knowledge, by longsuffering, by kindness, by the Holy Ghost, by love unfeigned,
  By the word of truth, by the power of God, by the armour of righteousness on the right hand and on the left,
  By honour and dishonour, by evil report and good report: as deceivers, and yet true;
  As unknown, and yet well known; as dying, and, behold, we live; as chastened, and not killed;
  As sorrowful, yet alway rejoicing; as poor, yet making many rich; as having nothing, and yet possessing all things. For we would not, brethren, have you ignorant of our trouble which came to us in Asia, that we were

In [191]:
summarization_pipeline_consolidation(corinthians2)

" Paul, an apostle of Jesus Christ by the will of God, and Timothy our brother, unto the church of God which is at Corinth, with all the saints which are in all Achaia:  Grace be to you and peace from God our Father, and from the Lord Jesus Christ . For as the sufferings of Christ abound in us, so our consolation also aboundeth by Christ . And whether we be afflicted, it is for your consolation and salvation, which is effectual in the enduring of the same sufferings which we also suffer . As also ye have acknowledged us in part, that we are your rejoicing, even as ye also are ours in the day of the lord Jesus Christ. But as God is true, our word toward you was not yea and nay? Our hope of you is stedfast, knowing,  We are troubled on every side, yet not distressed; we are perplexed, but not in despair;  Persecuted, though not forsaken; cast down, or cast down; cast away; ... And always bearing about in the body the dying of Jesus might be made manifest in our body . For in this we groa

## Galatians

In [113]:
galatians = get_book_text('Galatians', bible)

In [114]:
printing(galatians)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Paul, an apostle, (not of men, neither by man, but by Jesus Christ, and God the Father, who raised him from the dead;)
  And all the brethren which are with me, unto the churches of Galatia:
  Grace be to you and peace from God the Father, and from our Lord Jesus Christ,
  Who gave himself for our sins, that he might deliver us from this present evil world, according to the will of God and our Father:
  To whom be glory for ever and ever. It was added because of transgressions, till the seed should come to whom the promise was made; and it was ordained by angels in the hand of a mediator. He therefore that ministereth to you the Spirit, and worketh miracles among you, doeth he it by the works of the law, or by the hearing of faith? Have ye suffered so many things in vain? As many as desire to make a fair shew in the flesh, they constrain you to be circumcised; only lest they

In [194]:
summarization_pipeline_consolidation(galatians)

" Paul, an apostle, (not of men, neither by man, but by Jesus Christ, and God the Father, who raised him from the dead;) And all the brethren which are with me, unto the churches of Galatia: Grace be to you and peace from God . But though we, or an angel from heaven, preach any other gospel unto you than that which we have preached unto you, let him be accursed . For I neither received it of man, nor was I taught it, but  by the revelation of Jesus Christ . After three years I went up to Jerusalem to see Peter, and abode with James the Lord's brother . James, Cephas, and John, perceived the grace that was given unto me, they gave to me and Barnabas the right hands of fellowship .  Brethren, I beseech you, be as I am; for I am as ye are: ye have not injured me at all . Ye know how through infirmity of the flesh I preached the gospel unto you at the first . But now, after that ye have known God, how turn ye again to the weak and beggarly elements, whereunto ye desire again to be in bonda

## Ephesians

In [115]:
ephesians = get_book_text('Ephesians', bible)

In [116]:
printing(ephesians)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Wherefore I also, after I heard of your faith in the Lord Jesus, and love unto all the saints,
  Cease not to give thanks for you, making mention of you in my prayers;
  That the God of our Lord Jesus Christ, the Father of glory, may give unto you the spirit of wisdom and revelation in the knowledge of him:
  The eyes of your understanding being enlightened; that ye may know what is the hope of his calling, and what the riches of the glory of his inheritance in the saints,
  And what is the exceeding greatness of his power to us-ward who believe, according to the working of his mighty power,
  Which he wrought in Christ, when he raised him from the dead, and set him at his own right hand in the heavenly places,
  Far above all principality, and power, and might, and dominion, and every name that is named, not only in this world, but also in that which is to come:
  And hath 

In [197]:
summarization_pipeline_consolidation(ephesians)

' Paul, an apostle of Jesus Christ by the will of God, to the saints which are at Ephesus, and to the faithful in Christ Jesus: "Grace be to you, and peace, from God our Father, and from the Lord Jesus Christ" In whom also we have obtained an inheritance, being predestinated according to the purpose of him who worketh all things after the counsel of his own will," says Paul . The Bible says: "We are no more strangers and foreigners, but fellowcitizens with the saints, and  of the household of God" Unsearchable riches of Christ were intended to be known by the church . Unto me, who am less than the least of all saints, is this grace given, that I should preach among the Gentiles ... That I should prepare for you  Tychicus: We all come in the unity of the faith, and of the knowledge of the Son of God, unto a perfect man . Fornication, and all uncleanness, or covetousness, let it not be once named among you, as becometh saints;  Neither filthiness, nor foolish talking, nor jesting, which 

## Philippians

In [117]:
philippians = get_book_text('Philippians', bible)

In [118]:
printing(philippians)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Yea doubtless, and I count all things but loss for the excellency of the knowledge of Christ Jesus my Lord: for whom I have suffered the loss of all things, and do count them but dung, that I may win Christ,
  And be found in him, not having mine own righteousness, which is of the law, but that which is through the faith of Christ, the righteousness which is of God by faith:
  That I may know him, and the power of his resurrection, and the fellowship of his sufferings, being made conformable unto his death;
  If by any means I might attain unto the resurrection of the dead. But if I live in the flesh, this is the fruit of my labour: yet what I shall choose I wot not. Not that I speak in respect of want: for I have learned, in whatsoever state I am, therewith to be content. For the same cause also do ye joy, and rejoice with me. All the saints salute you, chiefly they that ar

In [200]:
summarization_pipeline_consolidation(philippians)

' Paul and Timotheus, the servants of Jesus Christ, to all the saints in Christ Jesus which are at Philippi, with the bishops and deacons: Grace be unto you, and peace . The one preach Christ of contention, not sincerely, supposing to add affliction to my bonds: but the other of love, knowing that I am set for the defence of the gospel. What then? notwithstanding, every way, whether in pretence, or in truth, Christ is preached; and I therein do rejoice, yea, and will rejoice. For now also Christ shall be magnified in my body, whether it be by life, or by death. For to me to live is Christ, and to die is gain. But if I live in the flesh, this is the fruit of my labour: yet what I shall choose I wot not  For I am in a strait betwixt two, having a desire to depart, and to be with Christ; which is far better . Nevertheless to abide in the flesh is more needful for you. And having this confidence, I know that I shall abide and continue with you all for your furtherance and joy of faith . Fu

## Colossians

In [119]:
colossians = get_book_text('Colossians', bible)

In [158]:
printing(colossians)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
For this cause we also, since the day we heard it, do not cease to pray for you, and to desire that ye might be filled with the knowledge of his will in all wisdom and spiritual understanding;
  That ye might walk worthy of the Lord unto all pleasing, being fruitful in every good work, and increasing in the knowledge of God;
  Strengthened with all might, according to his glorious power, unto all patience and longsuffering with joyfulness;
  Giving thanks unto the Father, which hath made us meet to be partakers of the inheritance of the saints in light:
  Who hath delivered us from the power of darkness, and hath translated us into the kingdom of his dear Son:
  In whom we have redemption through his blood, even the forgiveness of sins:
  Who is the image of the invisible God, the firstborn of every creature:
  For by him were all things created, that are in heaven, and that

In [203]:
summarization_pipeline_consolidation(colossians)

' Paul, an apostle of Jesus Christ by the will of God, and Timotheus our brother, are at Colosse: Grace be unto you, and peace . We give thanks to God and the Father of our Lord Jesus Christ, praying always for you, with all patience and longsuffering with joyfulness . For the hope which is laid up for you in heaven, whereof ye heard before in the word of the truth of the gospel; ... And we also, since the day we heard it, do not cease to pray for you. That ye might walk worthy of the Lord unto all pleasing, being fruitful in every good work, and increasing in the knowledge of God . By him he is the image of the invisible God, the firstborn of every creature: He is before all things, and by him all things consist  Paul: If ye continue in the faith grounded and settled, and be not moved away from the hope of the gospel, which ye have heard, and which was preached to every creature which is under heaven . And you, that were sometime alienated and enemies in your mind by wicked works, yet

## 1 Thessalonians

In [121]:
thessalonians1 = get_book_text('1 Thessalonians', bible)

In [122]:
printing(thessalonians1)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And the very God of peace sanctify you wholly; and I pray God your whole spirit and soul and body be preserved blameless unto the coming of our Lord Jesus Christ. But let us, who are of the day, be sober, putting on the breastplate of faith and love; and for an helmet, the hope of salvation. And ye became followers of us, and of the Lord, having received the word in much affliction, with joy of the Holy Ghost:
  So that ye were ensamples to all that believe in Macedonia and Achaia. Wherefore when we could no longer forbear, we thought it good to be left at Athens alone;
  And sent Timotheus, our brother, and minister of God, and our fellowlabourer in the gospel of Christ, to establish you, and to comfort you concerning your faith:
  That no man should be moved by these afflictions: for yourselves know that we are appointed thereunto. But we, brethren, being taken from you fo

In [206]:
summarization_pipeline_consolidation(thessalonians1)

' Paul, and Silvanus, and Timotheus, unto the church of the Thessalonians which is in God the Father and in the Lord Jesus Christ: Grace be unto you, and peace, from God our Father . We give thanks to God always for you all, making mention of you in our prayers;  Remembering without ceasing your work of faith, and labour of love, and patience of hope in the sight of God and our Father;  Knowing, brethren beloved, your election of God. For our gospel came not only in word only, but also in power, and in Holy Ghost, and assurance; as ye know what manner of men we were among you for your sake . And ye became followers of us, and of the Lord, having received the word in much affliction, with joy of the Holy Ghost  For ye remember, brethren, our labour and travail: for labouring night and day, because we preached unto you the gospel of God . Ye are witnesses, and God also, how holily and justly and unblameably we behaved ourselves among you that believe:  As ye know how we exhorted and comf

## 2 Thessalonians

In [123]:
thessalonians2 = get_book_text('2 Thessalonians', bible)

In [124]:
printing(thessalonians2)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Paul, and Silvanus, and Timotheus, unto the church of the Thessalonians in God our Father and the Lord Jesus Christ:
  Grace unto you, and peace, from God our Father and the Lord Jesus Christ. And then shall that Wicked be revealed, whom the Lord shall consume with the spirit of his mouth, and shall destroy with the brightness of his coming:
  Even him, whose coming is after the working of Satan with all power and signs and lying wonders,
  And with all deceivableness of unrighteousness in them that perish; because they received not the love of the truth, that they might be saved. Therefore, brethren, stand fast, and hold the traditions which ye have been taught, whether by word, or our epistle. For the mystery of iniquity doth already work: only he who now letteth will let, until he be taken out of the way. Amen. Yet count him not as an enemy, but admonish him as a brother.

In [209]:
summarization_pipeline_consolidation(thessalonians2)

" Paul, Silvanus, and Timotheus, wrote to Thessalonians: Grace unto you, and peace, from God our Father and the Lord Jesus Christ . We are bound to thank God always for you, brethren, as it is meet, because that your faith groweth exceedingly . So that we ourselves glory in you in the churches of God for your patience and faith in all your persecutions and tribulations that ye endure:  Which is a manifest token of the righteous judgment of God, that ye may be counted worthy of the kingdom of God . When he shall come to be glorified in his saints, and to be admired in all them that believe (because our testimony among you was believed) in that day. Let no man deceive you by any means: For that day shall not come, except there come a falling away first,  We are bound to give thanks alway to God for you, brethren beloved of the Lord, because God has from the beginning chosen you to salvation through sanctification of the Spirit and belief of the truth . And for this cause God shall send t

## 1 Timothy

In [125]:
timothy1 = get_book_text('1 Timothy', bible)

In [126]:
printing(timothy1)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
I give thee charge in the sight of God, who quickeneth all things, and before Christ Jesus, who before Pontius Pilate witnessed a good confession;
  That thou keep this commandment without spot, unrebukeable, until the appearing of our Lord Jesus Christ:
  Which in his times he shall shew, who is the blessed and only Potentate, the King of kings, and Lord of lords;
  Who only hath immortality, dwelling in the light which no man can approach unto; whom no man hath seen, nor can see: to whom be honour and power everlasting. Rebuke not an elder, but intreat him as a father; and the younger men as brethren;
  The elder women as mothers; the younger as sisters, with all purity. Likewise must the deacons be grave, not doubletongued, not given to much wine, not greedy of filthy lucre;
  Holding the mystery of the faith in a pure conscience. And let these also first be proved; then 

In [212]:
summarization_pipeline_consolidation(timothy1)

" Paul, an apostle of Jesus Christ by the commandment of God our Saviour, and Lord Jesus Christ, which is our hope; . Unto Timothy, my own son in the faith: Grace, mercy, and peace, from God our Father and Jesus Christ our Lord . As I besought thee to abide still at Ephesus, when I went into Macedonia, that thou mightest charge some that they teach no other doctrine . Neither give heed to fables and endless genealogies, which minister questions, rather than godly edifying which is in faith: so do . The law is not made for a righteous man, but for the lawless and disobedient, for the ungodly and for sinners, for unholy and profane . For murderers of fathers and murderers of mothers, for manslayers, for menstealers, for  I exhort that men pray every where, lifting up holy hands, without wrath and doubting . In like manner also that women adorn themselves in modest apparel, with shamefacedness and sobriety . But I suffer not a woman to teach, nor to usurp authority over the man, but to be

## 2 Timothy

In [127]:
timothy2 = get_book_text('2 Timothy', bible)

In [128]:
printing(timothy2)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Be not thou therefore ashamed of the testimony of our Lord, nor of me his prisoner: but be thou partaker of the afflictions of the gospel according to the power of God;
  Who hath saved us, and called us with an holy calling, not according to our works, but according to his own purpose and grace, which was given us in Christ Jesus before the world began,
  But is now made manifest by the appearing of our Saviour Jesus Christ, who hath abolished death, and hath brought life and immortality to light through the gospel:
  Whereunto I am appointed a preacher, and an apostle, and a teacher of the Gentiles. For the time will come when they will not endure sound doctrine; but after their own lusts shall they heap to themselves teachers, having itching ears;
  And they shall turn away their ears from the truth, and shall be turned unto fables. I thank God, whom I serve from my foref

In [215]:
summarization_pipeline_consolidation(timothy2)

' To Timothy, my dearly beloved son: Grace, mercy, and peace, from God the Father and Christ Jesus our Lord . I thank God, whom I serve from my forefathers with pure conscience, that without ceasing I have remembrance of thee in my prayers night and day;  Greatly desiring to see thee, being mindful of thy tears, that I may be filled with joy;  When I call to remembrance the unfeigned faith that is in thee, which dwelt first in thy grandmother Lois, and thy mother Eunice; and I am persuaded that in thee also . Be not ashamed of the testimony of our Lord, nor of me his prisoner: but be thou partaker of the afflictions of the gospel according to the power of God . All they which are in Asia be turned away from me  No man that warreth entangle himself with the affairs of this life; that he may please him who has chosen him to be a soldier . And if a man also strive for masteries, yet is he not crowned, except he strive lawfully . Study to shew thyself approved unto God, a workman that need

## Titus

In [129]:
titus = get_book_text('Titus', bible)

In [130]:
printing(titus)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
The aged women likewise, that they be in behaviour as becometh holiness, not false accusers, not given to much wine, teachers of good things;
  That they may teach the young women to be sober, to love their husbands, to love their children,
  To be discreet, chaste, keepers at home, good, obedient to their own husbands, that the word of God be not blasphemed. Wherefore rebuke them sharply, that they may be sound in the faith;
  Not giving heed to Jewish fables, and commandments of men, that turn from the truth. For the grace of God that bringeth salvation hath appeared to all men,
  Teaching us that, denying ungodliness and worldly lusts, we should live soberly, righteously, and godly, in this present world;
  Looking for that blessed hope, and the glorious appearing of the great God and our Saviour Jesus Christ;
  Who gave himself for us, that he might redeem us from all in

In [218]:
summarization_pipeline_consolidation(titus)

" Paul was a servant of God, and an apostle of Jesus Christ, according to the faith of God's elect, and the acknowledging of the truth which is after godliness . For this cause left I thee in Crete, that thou shouldest set in order the things that are wanting, and ordain elders in every city, as I had appointed thee: If any be blameless, the husband of one wife, having faithful children not accused of riot or unruly . For there are many unruly and vain talkers and deceivers, specially they of the circumcision: Whose mouths must be stopped, who subvert whole houses, teaching things which they ought not, for filthy lucre's sake . One of themselves, even a prophet of their own, said, The Cretians are alway liars, evil beasts, slow bellies. This witness is true  In all things shewing thyself a pattern of good works: in doctrine shewing uncorruptness, gravity, sincerity, and sound speech . Exhort servants to be obedient unto their own masters, and to please them well in all things . Not pur

## Philemon

In [131]:
philemon = get_book_text('Philemon', bible)

In [132]:
printing(philemon)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Paul, a prisoner of Jesus Christ, and Timothy our brother, unto Philemon our dearly beloved, and fellowlabourer,
  And to our beloved Apphia, and Archippus our fellowsoldier, and to the church in thy house:
  Grace to you, and peace, from God our Father and the Lord Jesus Christ. If thou count me therefore a partner, receive him as myself. Amen. If he hath wronged thee, or oweth thee ought, put that on mine account;
  I Paul have written it with mine own hand, I will repay it: albeit I do not say to thee how thou owest unto me even thine own self besides. But withal prepare me also a lodging: for I trust that through your prayers I shall be given unto you. For we have great joy and consolation in thy love, because the bowels of the saints are refreshed by thee, brother. Written from Rome to Philemon, by Onesimus a servant. There salute thee Epaphras, my fellowprisoner in Chr

In [221]:
summarization_pipeline_consolidation(philemon)

"Paul, a prisoner of Jesus Christ, and Timothy our brother, unto Philemon our dearly beloved, and fellowlabourer,\n  And to our beloved Apphia, and Archippus our fellowsoldier, and to the church in thy house:\n  Grace to you, and peace, from God our Father and the Lord Jesus Christ.\n  I thank my God, making mention of thee always in my prayers,\n  Hearing of thy love and faith, which thou hast toward the Lord Jesus, and toward all saints;\n  That the communication of thy faith may become effectual by the acknowledging of every good thing which is in you in Christ Jesus.\n  For we have great joy and consolation in thy love, because the bowels of the saints are refreshed by thee, brother.\n  Wherefore, though I might be much bold in Christ to enjoin thee that which is convenient,\n  Yet for love's sake I rather beseech thee, being such an one as Paul the aged, and now also a prisoner of Jesus Christ.\n  I beseech thee for my son Onesimus, whom I have begotten in my bonds:\n  Which in ti

## James

In [133]:
james = get_book_text('James', bible)

In [159]:
printing(james)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
let him call for the elders of the church; and let them pray over him, anointing him with oil in the name of the Lord:
  And the prayer of faith shall save the sick, and the Lord shall raise him up; and if he have committed sins, they shall be forgiven him. For if there come unto your assembly a man with a gold ring, in goodly apparel, and there come in also a poor man in vile raiment;
  And ye have respect to him that weareth the gay clothing, and say unto him, Sit thou here in a good place; and say to the poor, Stand thou there, or sit here under my footstool:
  Are ye not then partial in yourselves, and are become judges of evil thoughts? Behold, the husbandman waiteth for the precious fruit of the earth, and hath long patience for it, until he receive the early and latter rain. Even so the tongue is a little member, and boasteth great things. Likewise also was not Rahab 

In [224]:
summarization_pipeline_consolidation(james)

" James, a servant of God and of the Lord Jesus Christ, to the twelve tribes which are scattered abroad, greeting. My brethren, count it all joy when ye fall into divers temptations;  Knowing this, that the trying of your faith worketh patience . If any of you lack wisdom, let him ask of God, that giveth to all men liberally, and upbraideth not; and it shall be given him. For he that wavereth is like a wave of the sea driven with the wind and tossed. A double minded man is unstable in all his ways. Let the brother of low degree rejoice in that he is exalted:  But the rich, in that they are made low: because as the flower of the grass he shall pass away. For the sun is no sooner  If any man among you seem to be religious, and bridleth not his tongue, but deceiveth his own heart, this man's religion is vain . Pure religion and undefiled before God and the Father is this, To visit the fatherless and widows in their affliction, and to keep himself unspotted from the world . For if any be a

## 1 Peter

In [135]:
peter1 = get_book_text('1 Peter', bible)

In [162]:
printing(peter1)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
But and if ye suffer for righteousness' sake, happy are ye: and be not afraid of their terror, neither be troubled;
  But sanctify the Lord God in your hearts: and be ready always to give an answer to every man that asketh you a reason of the hope that is in you with meekness and fear:
  Having a good conscience; that, whereas they speak evil of you, as of evildoers, they may be ashamed that falsely accuse your good conversation in Christ. For for this cause was the gospel preached also to them that are dead, that they might be judged according to men in the flesh, but live according to God in the spirit. Unto you therefore which believe he is precious: but unto them which be disobedient, the stone which the builders disallowed, the same is made the head of the corner,
  And a stone of stumbling, and a rock of offence, even to them which stumble at the word, being disobedien

In [227]:
summarization_pipeline_consolidation(peter1)

' Peter, an apostle of Jesus Christ, to the strangers scattered throughout Pontus, Galatia, Cappadocia, Asia, and Bithynia: Grace unto you, and peace, be multiplied . The trial of your faith, being much more precious than of gold that perisheth, though it be tried with fire, might be found unto praise and honour and glory at the appearing of Lord Jesus Christ . As obedient children, not fashioning yourselves according to the former lusts in your ignorance: But as he which hath called you is holy, so be ye holy in all manner of conversation; because it is written, Be ye holy; for I am holy. To an inheritance incorruptible, and undefiled,   With the precious blood of Christ, as of a lamb without blemish and without spot: \'Being born again, not of corruptible seed, but of incorruptible, by the word of God, which liveth and abideth for ever. Forasmuch as ye know that ye were not redeemed with corruptible things, as silver and gold, from your vain conversation received by tradition from yo

## 2 Peter

In [137]:
peter2 = get_book_text('2 Peter', bible)

In [138]:
printing(peter2)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
Simon Peter, a servant and an apostle of Jesus Christ, to them that have obtained like precious faith with us through the righteousness of God and our Saviour Jesus Christ:
  Grace and peace be multiplied unto you through the knowledge of God, and of Jesus our Lord,
  According as his divine power hath given unto us all things that pertain unto life and godliness, through the knowledge of him that hath called us to glory and virtue:
  Whereby are given unto us exceeding great and precious promises: that by these ye might be partakers of the divine nature, having escaped the corruption that is in the world through lust. And through covetousness shall they with feigned words make merchandise of you: whose judgment now of a long time lingereth not, and their damnation slumbereth not. These are wells without water, clouds that are carried with a tempest; to whom the mist of dark

In [230]:
summarization_pipeline_consolidation(peter2)

' Simon Peter, a servant and an apostle of Jesus Christ, to them that have obtained like precious faith with us through the righteousness of God and our Saviour Jesus Christ: . By these ye might be partakers of the divine nature, having escaped the corruption that is in the world through lust . And beside this, giving all diligence, add to your faith virtue; and to virtue knowledge . And to godliness brotherly kindness charity. For if these things be in you, and abound, they make you that ye shall neither be barren nor unfruitful in the knowledge of our Lord Jesus Christ. But he that lacketh these things is blind, and cannot see afar off, and hath forgotten that he was purged from his old sins. Yea, I think it meet, as long as I am in this tabernacle, to stir you up  There were false prophets also among the people, even as there shall be false teachers among you, who privily shall bring in damnable heresies . And many shall follow their pernicious ways; by reason of whom the way of tru

## 1 John

In [139]:
john1 = get_book_text('1 John', bible)

In [140]:
printing(john1)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
That which was from the beginning, which we have heard, which we have seen with our eyes, which we have looked upon, and our hands have handled, of the Word of life;
  (For the life was manifested, and we have seen it, and bear witness, and shew unto you that eternal life, which was with the Father, and was manifested unto us;)
  That which we have seen and heard declare we unto you, that ye also may have fellowship with us: and truly our fellowship is with the Father, and with his Son Jesus Christ. We know that whosoever is born of God sinneth not; but he that is begotten of God keepeth himself, and that wicked one toucheth him not. And now, little children, abide in him; that, when he shall appear, we may have confidence, and not be ashamed before him at his coming. Herein is our love made perfect, that we may have boldness in the day of judgment: because as he is, so are 

In [233]:
summarization_pipeline_consolidation(john1)

" This then is the message which we have heard of him, and declare unto you, that God is light, and in him is no darkness at all . He that saith he abideth in him ought himself also so to walk, even as he walked . If we say that we have no sin, we deceive ourselves, and the truth is not in us. But whoso keepeth his word, in him verily is the love of God perfected: hereby know we that we are in him. And hereby we do know that we know him, if we keep his commandments. The old commandment is the word which ye have heard from the beginning. Again, a new commandment I write unto you,. which thing is true in him and in you: because the darkness is past,  the true light now shineth. He that hateth his brother  I write unto you, fathers, because ye have known him that is from the beginning . I write to you, little children, because your sins are forgiven you for his name's sake . But he that hateth his brother is in darkness and walketh in darkness, and knoweth not whither he goeth, because th

## 2 John

In [141]:
john2 = get_book_text('2 John', bible)

In [142]:
printing(john2)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
The elder unto the elect lady and her children, whom I love in the truth; and not I only, but also all they that have known the truth;
  For the truth's sake, which dwelleth in us, and shall be with us for ever. Grace be with you, mercy, and peace, from God the Father, and from the Lord Jesus Christ, the Son of the Father, in truth and love. I rejoiced greatly that I found of thy children walking in truth, as we have received a commandment from the Father. And now I beseech thee, lady, not as though I wrote a new commandment unto thee, but that which we had from the beginning, that we love one another. And this is love, that we walk after his commandments. This is the commandment, That, as ye have heard from the beginning, ye should walk in it. For many deceivers are entered into the world, who confess not that Jesus Christ is come in the flesh. This is a deceiver and an ant

In [236]:
summarization_pipeline_consolidation(john2)

"The elder unto the elect lady and her children, whom I love in the truth; and not I only, but also all they that have known the truth;\n  For the truth's sake, which dwelleth in us, and shall be with us for ever.\n  Grace be with you, mercy, and peace, from God the Father, and from the Lord Jesus Christ, the Son of the Father, in truth and love.\n  I rejoiced greatly that I found of thy children walking in truth, as we have received a commandment from the Father.\n  And now I beseech thee, lady, not as though I wrote a new commandment unto thee, but that which we had from the beginning, that we love one another.\n  And this is love, that we walk after his commandments. This is the commandment, That, as ye have heard from the beginning, ye should walk in it.\n  For many deceivers are entered into the world, who confess not that Jesus Christ is come in the flesh. This is a deceiver and an antichrist.\n  Look to yourselves, that we lose not those things which we have wrought, but that we

## 3 John

In [143]:
john3 = get_book_text('3 John', bible)

In [144]:
printing(john3)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
The elder unto the wellbeloved Gaius, whom I love in the truth. Beloved, I wish above all things that thou mayest prosper and be in health, even as thy soul prospereth. For I rejoiced greatly, when the brethren came and testified of the truth that is in thee, even as thou walkest in the truth. I have no greater joy than to hear that my children walk in truth. Beloved, thou doest faithfully whatsoever thou doest to the brethren, and to strangers;
  Which have borne witness of thy charity before the church: whom if thou bring forward on their journey after a godly sort, thou shalt do well:
  Because that for his name's sake they went forth, taking nothing of the Gentiles. We therefore ought to receive such, that we might be fellowhelpers to the truth. I wrote unto the church: but Diotrephes, who loveth to have the preeminence among them, receiveth us not. Wherefore, if I come,

In [239]:
summarization_pipeline_consolidation(john3)

"The elder unto the wellbeloved Gaius, whom I love in the truth.\n  Beloved, I wish above all things that thou mayest prosper and be in health, even as thy soul prospereth.\n  For I rejoiced greatly, when the brethren came and testified of the truth that is in thee, even as thou walkest in the truth.\n  I have no greater joy than to hear that my children walk in truth.\n  Beloved, thou doest faithfully whatsoever thou doest to the brethren, and to strangers;\n  Which have borne witness of thy charity before the church: whom if thou bring forward on their journey after a godly sort, thou shalt do well:\n  Because that for his name's sake they went forth, taking nothing of the Gentiles.\n  We therefore ought to receive such, that we might be fellowhelpers to the truth.\n  I wrote unto the church: but Diotrephes, who loveth to have the preeminence among them, receiveth us not.\n  Wherefore, if I come, I will remember his deeds which he doeth, prating against us with malicious words: and n

## Jude

In [145]:
jude = get_book_text('Jude', bible)

In [146]:
printing(jude)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
But ye, beloved, building up yourselves on your most holy faith, praying in the Holy Ghost,
  Keep yourselves in the love of God, looking for the mercy of our Lord Jesus Christ unto eternal life. These are murmurers, complainers, walking after their own lusts; and their mouth speaketh great swelling words, having men's persons in admiration because of advantage. These are spots in your feasts of charity, when they feast with you, feeding themselves without fear: clouds they are without water, carried about of winds; trees whose fruit withereth, without fruit, twice dead, plucked up by the roots;
  Raging waves of the sea, foaming out their own shame; wandering stars, to whom is reserved the blackness of darkness for ever. But these speak evil of those things which they know not: but what they know naturally, as brute beasts, in those things they corrupt themselves. for they 

In [242]:
summarization_pipeline_consolidation(jude)

"Jude, the servant of Jesus Christ, and brother of James, to them that are sanctified by God the Father, and preserved in Jesus Christ, and called:\n  Mercy unto you, and peace, and love, be multiplied.\n  Beloved, when I gave all diligence to write unto you of the common salvation, it was needful for me to write unto you, and exhort you that ye should earnestly contend for the faith which was once delivered unto the saints.\n  For there are certain men crept in unawares, who were before of old ordained to this condemnation, ungodly men, turning the grace of our God into lasciviousness, and denying the only Lord God, and our Lord Jesus Christ.\n  I will therefore put you in remembrance, though ye once knew this, how that the Lord, having saved the people out of the land of Egypt, afterward destroyed them that believed not.\n  And the angels which kept not their first estate, but left their own habitation, he hath reserved in everlasting chains under darkness unto the judgment of the gr

## Revelation

In [147]:
revelation = get_book_text('Revelation', bible)

In [148]:
printing(revelation)

****************************
*** TF-IDF Consolidation ***
****************************
Max pooling:
------------
And the third angel followed them, saying with a loud voice, If any man worship the beast and his image, and receive his mark in his forehead, or in his hand,
  The same shall drink of the wine of the wrath of God, which is poured out without mixture into the cup of his indignation; and he shall be tormented with fire and brimstone in the presence of the holy angels, and in the presence of the Lamb:
  And the smoke of their torment ascendeth up for ever and ever: and they have no rest day nor night, who worship the beast and his image, and whosoever receiveth the mark of his name. John to the seven churches which are in Asia: Grace be unto you, and peace, from him which is, and which was, and which is to come; and from the seven Spirits which are before his throne;
  And from Jesus Christ, who is the faithful witness, and the first begotten of the dead, and the prince of the

In [245]:
summarization_pipeline_consolidation(revelation)

' In the midst of the seven candlesticks one like unto the Son of man, clothed with a garment down to the foot, and girt about the paps with a golden girdle . He had in his right hand seven stars: and out of his mouth went a sharp twoedged sword . After this I looked, and, behold, a door was opened in heaven . There were seven lamps of fire burning before the throne are the seven Spirits of God . All the angels stood round about the throne, and about the elders and the four beasts, and worshipped God, saying, Amen . They shall hunger no more, neither thirst any more; neither shall the sun light on them, nor any heat . And there were voices, and thunderings, and lightnings, and an earthquake . To him that overcometh will  And thus I saw the horses in the vision, and them that sat on them, having breastplates of fire, and of jacinth, and brimstone . The heads of the horses were as the heads of lions; and out of their mouths issued fire and smoke and brimstones . By these three was the th